In [1]:
import os
import time 
import scipy
import numpy as np 
import matplotlib 
import matplotlib.pyplot as plt 
plt.style.use('./utils/tecplot.mplstyle')

import torch 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
import torch.utils as utils 
from torch.utils.data import Dataset, DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# import neuralop 
# from utilities import *
from utils.neuraloperator import * 

from pathlib import Path
from torch import Tensor
from typing import Any, List, Tuple, Mapping, Optional, Iterable, Union, Dict, Literal

## libraries for CFD
# import cupy as cp
import h5py 
import yaml

from tqdm import tqdm
import wandb

cuda


In [ ]:
config={
    'name': '2dHIT_UNet', 
    'architecture': 'UNet',
    'model': {
        'params': {
            'in_channels': 1, 
            'out_channels': 1, 
            'activation': 'GELU', # nn.LeakyReLU(0.2), 
            'pooling': nn.AvgPool2d(kernel_size=2, stride=2),
            'norm': 'instancenorm',
            'embedding_channels': 1024, 
            # 'channels': None, # 1,
        },
        'device': device,
    },
    
    'epochs': 50, 
    'optimizer': {
        'loss_fn': 'h1',
        'params': {
            'weight_decay': 5e-4, 
            'lr': 1e-4,
            # 'amp_autocast': False, 
        },

        'scheduler': {
            'name': 'ExponentialLR', 
            'params': {
                'gamma': 0.5, 
            },
        },
    },
    'data': [
        { 'nu': 5e-05, 'leadtime': 0.1, # 'idx_leadtime': 1, 
         'stage': 'fit',
        'params': {
            'base_path': '../Data/nu=5e-05_n=512_fDNS=128/', 
            'dataset_name': f'2dHIT_nu=5e-05_n=512_T=17.0_fDNS=128', 
            
            'n_input': 1, 
            'n_output':1, 
            'batch_size': 64,
            'num_workers': 0, 

            'Ndata_train':500,  
            'Ndata_val':30, 
        },
        'normalization': {
            'normalization_path': '../Data/nu=5e-05_n=512_fDNS=128/config.yaml',
        }
        },
    #     { 'nu': 5e-05, 'leadtime': 0.1, # 'idx_leadtime': 1, 
    #      'stage': 'valid',
    #     'params': {
    #         'base_path': '../Data/nu=5e-05_n=512/', 
    #         'dataset_name': f'2dHIT_nu=5e-05_n=512_T=17.0', 
            
    #         'n_input': 1, 
    #         'n_output':1, 
    #         'batch_size': 64,
    #         'num_workers': 0,

    #         'Ndata_val':1,  
    #     },
    #     'normalization': {
    #         'normalization_path': '../Data/nu=5e-05_n=512/config.yaml',
    #     }
    # },
    ]
    
}

In [3]:
### CFD equation parameters ###
integral_timescales = {
    1e-3: 1.870162606239319,
    5e-4: 1.5808453559875488,
    1e-4: 1.3038111189,
    5e-05: 1.15829598903656, 
}

In [4]:
def calc_rms(vel:torch.Tensor, 
             dim:int=2,
             ) -> torch.Tensor: 
    """
    Compute the root mean square. 


    """
    if not dim: 
        dim = vel.ndim - 1
        vel = vel.unsqueeze(0) 
    batch_shape, channels, shape = vel.shape[:-(dim+1)], vel.shape[-(dim+1)], vel.shape[-dim:] 
    fft_dim = tuple(range(-dim, 0))

    vel_fluc = vel - vel.mean(dim=fft_dim, keepdim=True)
    vel_rms = vel_fluc.pow(2).mean(dim=fft_dim)
    vel_rms = torch.sqrt(vel_rms.mean(dim=-1))
    return vel_rms

def vor2vel(vor: torch.Tensor, 
            dim: int=None, 
            ) -> torch.Tensor:
    """
    Convert vorticity field to vector field.
        𝑢̂(𝑘) = (𝑖 · (𝑘 × 𝜔̂(𝑘)))⁄∣𝑘∣²
        
    Parameters:
    - vor (torch.tensor): vorticity field, shape (batch_shape, 1, Nx, Ny) for 2d or (batch_shape, 3, Nx, Ny, Nz) for 3d. 
    - dim (int): dimension of the field. 
    
    Returns:
    - vec (torch.tensor): vector field, shape (batch_shape, 2, Nx, Ny) for 2d or (batch_shape, 3, Nx, Ny, Nz) for 3d. 
    """
    if not dim: 
        dim = vor.ndim - 1
        vor = vor.unsqueeze(0) 
    batch_shape, channels, shape = vor.shape[:-(dim+1)], vor.shape[-(dim+1)], vor.shape[-dim:] 

    ## Compute the wavenumber 
    k = [torch.fft.fftfreq(n, 1/n, device=vor.device) for n in shape] 
    k = torch.meshgrid(k, indexing='ij')
    k = torch.stack(k)
    k2 = torch.sum(k**2, axis=0); k2[tuple(0 for _ in k2.shape)] = 1

    ## Compute velocity field
    fft_dim = tuple(range(-dim, 0)) 
    vor_k = torch.fft.fftn(vor, dim=fft_dim, norm='forward') # forard norm. refer: Pope (2000) p.678
    
    if dim == 2: 
        cross = torch.concat([k[1] * vor_k, - k[0] * vor_k], dim=len(batch_shape))
    else: 
        k = k.to(vor_k.dtype).unsqueeze(0)
        cross = torch.cross(k, vor_k, dim=1)
    k2 = k2.unsqueeze(0).unsqueeze(0) 
    
    vel_k = 1j * cross / k2 ## 𝑢̂(𝑘) = (𝑖 · (𝑘 × 𝜔̂(𝑘)))⁄∣𝑘∣²
    vel = torch.fft.ifftn(vel_k, dim=fft_dim, norm='forward').real
    return vel

def calc_tke(
        vor:torch.Tensor=None,
        vel:torch.Tensor=None, 
        dim:int=2, 
        ) -> torch.Tensor:
    """
    Calculate the turbulent kinetic energy (TKE) from velocity field.
        TKE = 0.5 * ⟨u² + v² + w²⟩
    
    Parameters:
    - vel (torch.tensor): velocity field, shape (B, 2, Nx, Ny) for 2d or (B, 3, Nx, Ny, Nz) for 3d.
    - dim (int): dimension of the field. 

    Returns:
    - tke (torch.tensor): TKE, shape (B,) 
    """
    if vor is not None: vel = vor2vel(vor, dim=dim)
    if not dim or dim == vel.ndim - 1: 
        dim = vel.ndim - 1
        vel = vel.unsqueeze(0) 
    batch_shape, channels, shape = vel.shape[:-(dim+1)], vel.shape[-(dim+1)], vel.shape[-dim:] 
    
    fft_dim = tuple(range(-dim, 0))
    tke = 0.5 * torch.mean((vel - torch.mean(vel, dim=fft_dim, keepdims=True))**2, dim=(-(dim+1),*fft_dim))
    return tke

def calc_dissipation(
        nu:float,
        vel:torch.Tensor=None, 
        vor:torch.Tensor=None,
        dim:int=2, 
        ) -> torch.Tensor:
    """
    Calculate the dissipation rate from vorticity and velocity field.
        for 3d, ε = 2 * ν * ⟨sᵢⱼ * sᵢⱼ⟩ 
                sᵢⱼ = 0.5 * (duᵢ/dxⱼ + duⱼ/dxᵢ)
        for 3d HIT, ⟨sᵢⱼ * sᵢⱼ⟩ = (S11² + S22² + S33² + 2*(0.5*(S12+S21))² + 2*(0.5*(S13+S31))² + 2*(0.5*(S23+S32))²)
        for 2d HIT, ε = 2 * ν * Ω
    Parameters:
    - nu (float): kinematic viscosity.
    - vel (torch.tensor): velocity field, shape (B, 2, Nx, Ny) for 2d or (B, 3, Nx, Ny, Nz) for 3d.
    - vor (torch.tensor): vorticity field, shape (B, 1, Nx, Ny) for 2d or (B, 3, Nx, Ny, Nz) for 3d. 
    - dim (int): dimension of the field. 

    Returns:
    - dissipation (torch.tensor): dissipation rate, shape (B,) if reduction is 'mean' or (B, 1, Nx, Ny) for 2d or (B, 1, Nx, Ny, Nz) for 3d.
    """
    if not dim:
        if vel is not None: 
            dim = vel.ndim - 1
            vel = vel.unsqueeze(0) 
        else: 
            dim = vor.ndim - 1
            vor = vor.unsqueeze(0) 
    batch_shape, channels, shape = (
        (vel.shape[:-(dim+1)], vel.shape[-(dim+1)], vel.shape[-dim:])
        if vel is not None
        else (vor.shape[:-(dim+1)], vor.shape[-(dim+1)], vor.shape[-dim:])
    )
    fft_dim = tuple(range(-dim, 0))
    
    if dim == 2:
        enstrophy = 0.5 * (vor - torch.mean(vor, dim=fft_dim, keepdim=True))**2
        ## dissipation rate field for 2d HIT ε = 2 * ν * Ω
        dissipation = 2 * nu * torch.mean(enstrophy, dim=fft_dim).mean(dim=-1)

    elif dim == 3:
        if vor is not None: vel = vor2vel(vor, dim=dim)
        ## Compute the wavenumber 
        k = [torch.fft.fftfreq(n, 1/n, device=vel.device) for n in shape] 
        k = torch.meshgrid(k, indexing='ij')
        k = torch.all(k)

        ## Compute velocity gradient
        vel_k = torch.fft.fftn(vel, dim=fft_dim, norm='forward') # forard norm. refer: Pope (2000) p.678        
        vel_grad_k = torch.all([1j * k[i].unsqueeze(0).unsqueeze(0) * vel_k for i in range(dim)], dim=1) # (B, 3(i), 3(j), Nx, Ny, Nz)
        vel_grad = torch.fft.ifftn(vel_grad_k, dim=fft_dim, norm='forward').real
        u_grad, v_grad, w_grad = vel_grad.unbind(dim=1) # (B, 3(j), Nx, Ny, Nz)
        u_x, u_y, u_z = u_grad.unbind(dim=1) # (B, Nx, Ny, Nz)
        v_x, v_y, v_z = v_grad.unbind(dim=1)    
        w_x, w_y, w_z = w_grad.unbind(dim=1)
        ## Stain rate tensor
        s_ij2 = (
            u_x**2 + v_y**2 + w_z**2 # S11, S22, S33
            + 2 * (0.5 * (u_y + v_x))**2 # S12
            + 2 * (0.5 * (v_z + w_y))**2 # S23
            + 2 * (0.5 * (u_z + w_x))**2 # S13
        ).unsqueeze(1)
        ## dissipation rate field 
        dissipation = 2 * nu * torch.mean(s_ij2, dim=fft_dim)
    return dissipation

def calc_Kolmogorov_lengthscale(dissipation:Union[torch.Tensor, float],
                                nu:float, 
                                ) -> Union[torch.Tensor, float]:
    """
    Compute the Kolmogorov lengthscale 
        η = ν^(3/4)/ε^(1/4)
    
    Parameters 
    - dissipation (torch.Tensor or float): dissipation rate. shape (B, )
    - nu (float): kinematic viscosity

    Returns: 
    - eta (torch.Tensor or float): Kolmogorov lenghscale. shape (B, )
    """
    return nu**(3/4) / dissipation**(1/4)

def calc_k(shape:tuple) -> torch.Tensor:
    '''
    Compute the wavenumbers from the shape of a 2D or 3D grid

    Parameters: 
    - shape (Tuple): The shape of the grid. (Nx, Ny) for 2D or (Nx, Ny, Nz) for 3D

    Returns: 
    - k (torch.tensor): The wavenumbers, shape (Nx, Ny) for 2D or (Nx, Ny, Nz) for 3D
    '''
    global k
    if 'k' in globals(): return k
    k = [torch.fft.fftfreq(n, 1/n) for n in shape] 
    k = torch.meshgrid(k, indexing='ij')
    k = torch.stack(k)
    # k = torch.sqrt(torch.sum(k**2, axis=0)) # tip: faster than torch.sum.
    
    return k

def calc_energy_spectrum(vel: torch.Tensor, dim:int=2) -> torch.Tensor:
    '''
    Compute the energy spectrum from a 2D or 3D velocity field
        for 3d, E(k) = ⟨|u(k)|²⟩     
        for 2d, E(k) = π k ⟨|u(k)|²⟩ 
    Parameters: 
    - vel (torch.tensor): The velocity field. 
        shape:  
            (B, 2, Nx, Ny) for 2d velocity field (with batch)
            (B, 3, Nx, Ny, Nz) for 3d velocity field
            (B, 1, Nx, Ny) for 2d vorticify field (enstrophy spectrum)
            (B, 3, Nx, Ny, Nz) for 3d vorticity field (enstrophy spectrum)
    - dim (int): The dimension of the velocity field. if None, the dimension is inferred from the shape of vel

    Returns: 
    - spectrum (torch.tensor): The energy spectrum. shape (B, Nk,) if batch, or (Nk,)
    '''
    if not dim: 
        dim = vel.ndim - 1
        vel = vel.unsqueeze(0) 
    batch_shape, channels, shape = vel.shape[:-(dim+1)], vel.shape[-(dim+1)], vel.shape[-dim:] 

    ## Compute the wavenumbers
    k = [torch.fft.fftfreq(n, 1/n, device=vel.device) for n in shape] 
    k = torch.meshgrid(k, indexing='ij')
    k = torch.stack(k)
    k = torch.sqrt(torch.sum(k**2, axis=0)) 
    Nk = min(shape)//2 # int(torch.ceil(k.max())) 

    ## Compute the energy spectrum
    fft_dim = tuple(range(-dim, 0)) 
    vel_k = torch.fft.fftn(vel, dim=fft_dim, norm='forward') # forard norm. refer: Pope (2000) p.678
    if dim == 2: 
        ## E(k) = π k ⟨|u(k)|²⟩ for 2d spectrum 
        E_k = 0.5 * (np.pi * k * torch.abs(vel_k)**2).sum(dim=-(dim+1)) # tip: faster than torch.real(vel_k * torch.conj(vel_k))
    else: 
        ## E(k) = ⟨|u(k)|²⟩ for 3d spectrum
        E_k = 0.5 * (torch.abs(vel_k)**2).sum(dim=-(dim+1)) 
    
    ## Compute index
    idx = torch.floor(k + 0.5).long()
    idx = torch.where((idx > Nk) | (idx < 0), torch.zeros_like(idx), idx)
    
    ## tips: scatter_add_ is faster than loop
    spectrum = torch.zeros((*batch_shape, Nk + 1), device=vel.device)
    idx = idx.view(-1)[(None,) * len(batch_shape)]
    spectrum.scatter_add_(-1, 
                          index =idx.expand(*batch_shape, -1), 
                          src=E_k.view(*batch_shape, -1),
                          )
    
    if batch_shape == 1: spectrum = spectrum.squeeze(0)
    return spectrum

def calc_enstrophy_spectrum(vor, dim:int=2):
    return calc_energy_spectrum(vor, dim=dim)

def calc_taylor_lengthscale(
        vel:torch.Tensor=None, 
        vor:torch.Tensor=None,
        dim:int=2, 
        nu: float=None, 
        dissipation: Union[torch.Tensor, float]=None,
        ) -> torch.Tensor:
    """
    Compute the Taylor lengthscale.
        for 2d, λ = (uᵣₘₛ / ωᵣₘₛ)^0.5
        for 3d, λ = uᵣₘₛ * (∂ u₁/∂ x₁)^{-0.5}
                  = (15 * ν/ε)^0.5 * uᵣₘₛ
    Parameters:
    - vel (torch.tensor): velocity field, shape (B, 2, Nx, Ny) for 2d or (B, 3, Nx, Ny, Nz) for 3d.
    - vor (torch.tensor): vorticity field, shape (B, 1, Nx, Ny) for 2d or (B, 3, Nx, Ny, Nz) for 3d. 
    - dim (int): dimension of the field. 
    - nu (float): kinematic viscosity
    - dissipation (torch.tensor): dissipation rate, shape (B, )

    Returns:
    - taylor_lengthscale (torch.tensor): taylor lengthscale, shape (B,)

    Reference:
    - Boffetta, G., & Ecke, R. E. (2011). Two-Dimensional Turbulence.
    """
    if not dim:
        if vel is not None: 
            dim = vel.ndim - 1
            vel = vel.unsqueeze(0) 
        else: 
            dim = vor.ndim - 1
            vor = vor.unsqueeze(0) 
    batch_shape, channels, shape = (
        (vel.shape[:-(dim+1)], vel.shape[-(dim+1)], vel.shape[-dim:])
        if vel is not None
        else (vor.shape[:-(dim+1)], vor.shape[-(dim+1)], vor.shape[-dim:])
    )
    fft_dim = tuple(range(-dim, 0))

    if dim == 2: 
        if vel is None: vel = vor2vel(vor, dim=2)
        u_rms = calc_rms(vel, dim=dim) # torch.sqrt(torch.mean((vel - vel.mean(dim=fft_dim, keepdim=True)).pow(2), dim=fft_dim, keepdim=True).mean(dim=-1, keepdim=True))
        w_rms = calc_rms(vor, dim=dim) # torch.std(vor, dim=fft_dim)
        taylor_lengthscale = (u_rms / w_rms) ** 0.5 
    elif dim == 3:
        pass
    return taylor_lengthscale

def calc_Re_lambda(
        vel:torch.Tensor=None, 
        vor:torch.Tensor=None,
        dim:int=2, 
        nu: float=None, 
        taylor_lengthscale: Union[torch.Tensor, float]=None,
        ) -> torch.Tensor:
    """
    Compute the Taylor-scale Reynolds number Re_λ.
        Re_λ = uᵣₘₛ * λ / ν

    Parameters:
    - vel (torch.tensor): velocity field, shape (B, 2, Nx, Ny) for 2d or (B, 3, Nx, Ny, Nz) for 3d.
    - vor (torch.tensor): vorticity field, shape (B, 1, Nx, Ny) for 2d or (B, 3, Nx, Ny, Nz) for 3d. 
    - dim (int): dimension of the field. 
    - nu (float): kinematic viscosity
    - taylor_lengthscale (torch.tensor): taylor lengthscale, shape (B, )

    Returns:
    - Re_lambda (torch.tensor): taylor-scale Reynolds number, shape (B,)

    Reference:
    - Boffetta, G., & Ecke, R. E. (2011). Two-Dimensional Turbulence.
    """
    if not dim:
        if vel is not None: 
            dim = vel.ndim - 1
            vel = vel.unsqueeze(0) 
        else: 
            dim = vor.ndim - 1
            vor = vor.unsqueeze(0) 
    batch_shape, channels, shape = (
        (vel.shape[:-(dim+1)], vel.shape[-(dim+1)], vel.shape[-dim:])
        if vel is not None
        else (vor.shape[:-(dim+1)], vor.shape[-(dim+1)], vor.shape[-dim:])
    )
    fft_dim = tuple(range(-dim, 0))

    if vel is None: vel = vor2vel(vor, dim=dim)
    if taylor_lengthscale is None: taylor_lengthscale = calc_taylor_lengthscale(vel=vel, vor=vor, nu=nu, dim=dim)
    u_rms = calc_rms(vel, dim=dim)
    return u_rms * taylor_lengthscale / nu

def calc_pdf(u, bins=50, range=(-3, 3), dim:int=2):
    bins = min(sum(u.shape), bins)
    # hist, bin_edges = torch.histogram(u, bins=bins, range=range, density=True)
    hist = torch.histc(u, bins=bins, min=range[0], max=range[1])
    bin_edges = torch.linspace(*range, bins+1, device=u.device)
    hist = hist / (torch.sum(hist) * torch.diff(bin_edges))

    bin_edges = (bin_edges[:-1] + bin_edges[1:]) / 2 # bin centers
    return torch.stack([bin_edges, hist], dim=0)

In [5]:
### DL ploter & logger 
def plot_2d_surface(u1, u2=None, axes=None, **kwargs):
    '''
    Parameters:
     - u1 (float np.array): field of t=t1 (1, Nx, Ny)
     - u2 (float np.array): field of t=t2 (1, Nx, Ny)
    '''
    if u2 is None: u2 = []
    if not isinstance(u2, list) and u2.ndim == 2: u2 = [u2]
    n = 1 + len(u2) if u2 is not None else 1
    row = min(n, 5) 
    col = n // 5 + 1
    if axes is None:
        fig = plt.figure(figsize=kwargs.get('figsize', (4*row, 4*col))) 
        ax1 = fig.add_subplot(col, row, 1)
        ax2 = [fig.add_subplot(col, row, i+2) for i in range(n - 1)]
    else:
        ax1, ax2 = axes
    vmax = kwargs.get('vmax', u1.max())
    vmin = kwargs.get('vmin', u1.min())

    ## plot field u1
    if torch.is_tensor(u1): u1 = u1.detach().cpu().numpy()
    cm1 = ax1.imshow(u1, origin='lower', aspect='auto',
                     vmax=vmax, vmin=vmin, 
                     )
    ax1.set(
        title=kwargs['title1'] if 'title1' in kwargs else kwargs['title'][0] if 'title' in kwargs else 'ground truth',
        xticks=[], yticks=[], 
    )
    if kwargs.get('cm1', False): plt.colorbar(cm1)

    ## plot field u2
    if torch.is_tensor(u2): u2 = u2.detach().cpu().numpy()
    elif isinstance(u2, list) and torch.is_tensor(u2[0]): u2 = [u.detach().cpu().numpy() for u in u2]
    for i in range(len(u2)):
        cm2 = ax2[i].imshow(u2[i], origin='lower', aspect='auto', 
                         vmax=vmax, vmin=vmin, 
                         )
        ax2[i].set(
            title=kwargs['title2'] if 'title2' in kwargs else kwargs['title'][i+1] if 'title' in kwargs else 'prediction',
            xticks=[], yticks=[], 
        )
        if kwargs.get('cm2', False): plt.colorbar(cm2)
    plt.suptitle(kwargs.get('suptitle', ''))
    plt.tight_layout()

def plot_spectrum(spectrum1, spectrum2=None, ax=None, **kwargs):
    '''
    plot spectrum in log-log plot 
    Nk: number of wavenumbers = resolution // 2

    Params:
    - spectrum1(1d array):            spectrum data 
    - (optional) spectrum2(1d array): spectrum data. if None, plot only spectrum1 and do not plot legend. 
    - (optional) ax(matplotlib.axes): plotting axes. if None, make new figure and show
    - (optional) kwargs(dict)
        - 'label1'(string):    label of spectrum 1 (default: 'target')
        - 'label2'(string):    label of spectrum 2 (default: 'prediction')
        - 'ylim'(tuple, (2,)): y-axis range (vmin, vmax)
        - 'title'(string):     plot title
    Assumptions:
    - spectrum1 and spectrum2 has same dimension.
    '''
    spectrum2 = [] if spectrum2 is None else [spectrum2] if not isinstance(spectrum2, list) else spectrum2

    Nk = len(spectrum1)
    k_1d = np.arange(Nk)

    if ax is None:
        fig = plt.figure(figsize=kwargs.get('figsize', (3, 3)))
        ax = fig.add_subplot()
    
    ## plot spectrum 1
    if torch.is_tensor(spectrum1): spectrum1 = spectrum1.detach().cpu().numpy()
    ax.plot(k_1d[1:], spectrum1[1:], 
            label=kwargs.get('label1', 'target')
            )

    ## plot spectrum 2
    if torch.is_tensor(spectrum2): spectrum2 = spectrum2.detach().cpu().numpy()
    elif isinstance(spectrum2, list) and spectrum2 and torch.is_tensor(spectrum2[0]): spectrum2 = [s.detach().cpu().numpy() for s in spectrum2]
    for i, spec in enumerate(spectrum2):
        Nk = len(spec)
        k_1d = np.arange(Nk)
        ax.plot(k_1d[1:], spec[1:], label=kwargs.get(f'label{i+2}', 'prediction'))

    Nk = max([len(spectrum1)] + [len(s) for s in spectrum2])
    
    
    ax.set(xscale='log', yscale='log', 
           xticks = [2 ** (np.round(np.log2(Nk)).astype(np.int32) - i) for i in range(min(np.round(np.log2(Nk)).astype(np.int32), 7))],
        )
    if 'ylim' in kwargs: ax.set_ylim(kwargs['ylim'])
    if 'title' in kwargs: ax.set_title(kwargs['title'])
    if spectrum2: ax.legend(loc=0)
    ax.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())

def plot_pdf(bin1, pdf1, bin2=None, pdf2=None, ax=None, **kwargs):
    '''
    plot pdf 

    Params:
    - pdf1(1d array): pdf data 
    - bin1: bin of pdf1
    - (optional) pdf2(1d array):      pdf data. 
    - (optional) bin2(1d array):      bin of pdf2. if None, plot only pdf1 and do not plot legend. 
    - (optional) ax(matplotlib.axes): plotting axes. if None, make new figure and show
    - (optional) kwargs(dict)
        - 'label1'(string): label of spectrum 1 (default: 'target')
        - 'label2'(string): label of spectrum 2 (default: 'prediction')
        - 'title'(string):  plot title
    Assumptions:
    - pdf1 and pdf2 has same dimension.
    '''
    if bin2 is not None and type(bin2) is not list:
        bin2 = [bin2]
        pdf2 = [pdf2]

    if ax is None:
        fig = plt.figure(figsize=kwargs.get('figsize', (3, 3)))
        ax = fig.add_subplot()

    ## plot pdf1
    if torch.is_tensor(pdf1): 
        bin1 = bin1.detach().cpu().numpy()
        pdf1 = pdf1.detach().cpu().numpy()
    ax.plot(bin1, pdf1, label=kwargs.get('label1', 'target'))

    ## plot pdf2
    if torch.is_tensor(pdf2): 
        bin2 = bin2.detach().cpu().numpy()
        pdf2 = pdf2.detach().cpu().numpy()
    elif isinstance(pdf2, list) and torch.is_tensor(pdf2[0]): 
        bin2 = [s.detach().cpu().numpy() for s in bin2]
        pdf2 = [s.detach().cpu().numpy() for s in pdf2]
    for i in range(len(bin2)):
        ax.plot(bin2[i], pdf2[i], label=kwargs.get(f'label{i+2}', 'prediction'))
    ax.set(yscale='log')

    if 'title' in kwargs: ax.set_title(kwargs['title'])
    if bin2 is not None:  ax.legend(loc=0)

In [6]:
import os
import numpy as np 
import torch 
import h5py
import glob
import time
from typing import Optional, Sequence

def high2low_box(
    x: torch.Tensor,
    scale_factors: Union[float, Sequence[float]] = 0.5,
    *,
    keepdim: bool = False,
    dim: int = 2,
    fft_norm: str = "backward",
    last_var_axis: bool = False,
) -> torch.Tensor:
    """Fourier‑domain box‑style scaling (down / up sampling).

    The function supports tensors shaped either as
        • (batch, channels, *spatial_dims)
        • (batch, channels, *spatial_dims, num_var)

    In the second case the last axis (``num_var``) is *not* included in the FFT
    and therefore remains unchanged.
    """

    if isinstance(scale_factors, float):
        scale_factors = [scale_factors] * dim
    scale_factors = list(scale_factors)
    if len(scale_factors) != dim:
        raise ValueError("`scale_factors` length must equal `dim`.")

    if x.ndim < dim + 2:
        raise ValueError(
            f"Input tensor must have at least {dim + 2} dimensions (got {x.ndim})."
        )

    if last_var_axis:
        spatial_axes = list(range(-(dim + 1), -1))
    else:
        spatial_axes = list(range(-dim, 0))
    spatial_shape = [x.shape[i] for i in spatial_axes]

    sf_tensor = torch.tensor(scale_factors, dtype=x.dtype, device=x.device)
    if torch.allclose(sf_tensor, torch.ones_like(sf_tensor)):
        return x.clone()

    Fx = torch.fft.fftn(x, dim=spatial_axes, norm=fft_norm)

    if (sf_tensor < 1).any():
        Fx = torch.fft.fftshift(Fx, dim=spatial_axes)

        if keepdim:
            mask = torch.ones_like(Fx, dtype=torch.bool)
            for rel_ax, (n, sf) in enumerate(zip(spatial_shape, scale_factors)):
                trim = int(round((1 - sf) * n / 2))
                if trim == 0:
                    continue
                abs_ax = spatial_axes[rel_ax] % Fx.ndim  # positive index
                low = torch.arange(trim, device=x.device)
                high = torch.arange(n - trim, n, device=x.device)
                mask.index_fill_(abs_ax, low, False)
                mask.index_fill_(abs_ax, high, False)
            Fx = Fx * mask
        else:
            slices = [slice(None)] * Fx.ndim
            for rel_ax, (n, sf) in enumerate(zip(spatial_shape, scale_factors)):
                trim = int(round((1 - sf) * n / 2))
                slices[spatial_axes[rel_ax]] = slice(trim, n - trim)
            Fx = Fx[tuple(slices)]

        Fx = torch.fft.ifftshift(Fx, dim=spatial_axes)

    # ----------------------------- up‑sampling branch -------------------------
    if (sf_tensor > 1).any():
        Fx = torch.fft.fftshift(Fx, dim=spatial_axes)

        # F.pad pads the *last* k dims; build list accordingly
        last_axes = list(range(-len(spatial_axes) - (1 if last_var_axis else 0), 0))
        pads: List[int] = []
        for ax in Reflectiond(last_axes):
            if ax in spatial_axes:
                rel = spatial_axes.index(ax)
                n = spatial_shape[rel]
                sf = scale_factors[rel]
                extra = int(round((sf - 1) * n))
                pads.extend([extra // 2, extra - extra // 2])
            else:  # var axis → no pad
                pads.extend([0, 0])
        Fx = F.pad(Fx, pads, mode="constant", value=0.0)
        Fx = torch.fft.ifftshift(Fx, dim=spatial_axes)

    # --------------------------------------------------- inverse FFT ----------
    x_out = torch.fft.ifftn(Fx, dim=spatial_axes, norm=fft_norm).real

    # ------------------------------ match statistics --------------------------
    def _mom(t: torch.Tensor):
        mean = t.mean(dim=spatial_axes, keepdim=True)
        std = t.std(dim=spatial_axes, unbiased=False, keepdim=True).clamp_min(1e-12)
        return mean, std

    mean_in, std_in = _mom(x)
    mean_out, std_out = _mom(x_out)

    x_out = (x_out - mean_out) / std_out * std_in + mean_in
    return x_out

class HIT2dDataset(Dataset):
    def __init__(self, 
        path: Optional[str] = None,
        base_path: Optional[str] = None,
        dataset_name: Optional[str] = None,
        split_name: Optional[str] = None,
        
        normalization:Optional[callable] = None,
        transform:Optional[callable] = None,

        n_input: int = 1,
        n_output: int = 1,
        n_stride: int = 0,
        max_rollout_steps=100,
        max_n_sim: Optional[int] = None,
        # batch_size:int=32, 
        
        # num_iteration_per_data:int=None, 
        # isDataAugmentation:bool=False,
        load_data:bool=True,
        verbose:bool=True,
        **kwargs, 
        ):
        super().__init__()
        self.path = path
        self.base_path = base_path
        self.dataset_name = dataset_name
        self.split_name = split_name
        if not path: 
            path = os.path.join(self.base_path, self.split_name, self.dataset_name) # f"{self.base_path}/{self.split_name}/{self.dataset_name}*"
        self.path = sorted(glob.glob(f"{path}*"))
        assert self.path, f"Error: Dataset path {path} does not exist."

        self.normalization = normalization
        self.transform = transform

        self.max_rollout_steps = max_rollout_steps
        self.n_input = n_input
        self.n_output = n_output
        self.n_stride = n_stride
        self.max_n_sim = max_n_sim if max_n_sim else np.inf
        
        self.verbose=verbose
        self.kwargs = kwargs

        self._build_metadata()
        self._calc_len()
        self.load_data = load_data
        if load_data:
            self._load_data()

    def _calc_len(self):
        self.n_sim = min(self.max_n_sim, sum(self.n_sim_per_file)) # self.n_sim = sum(self.n_sim_per_file) # len(self.data)
        self.n_steps_per_sim = self.Nt
        self.n_windows_per_sim = self.n_steps_per_sim - (self.n_input + self.n_output + self.n_stride) + 1
        self.len = self.n_sim * self.n_windows_per_sim

    def __len__(self):
        return self.len
    
    def __getitem__(self, idx:int):
        data = self._load_one_sample(idx)
        data = self._preprocess_data(data)
        if self.transform:
            data = self.transform(data)
        if self.normalization:
            data = self.normalization(data)      
        data = self._postprocess_data(data)
        return data
    
    def _load_one_sample(self, idx:int):
        isim = idx // self.n_windows_per_sim
        it = idx % self.n_windows_per_sim
        if self.load_data:
            data = self.data[isim, it:it + self.n_input] # (n_in, channels, *datashape)
            target = self.data[isim, it + self.n_input + self.n_stride:it + self.n_input + self.n_stride + self.n_output] # (n_out, channels, *datashape)
        else: 
            for i in range(len(self.n_sim_per_file)):
                ifile = i 
                if isim < sum(self.n_sim_per_file[:i+1]): break 
            
            isim = isim - sum(self.n_sim_per_file[:i])
            with h5py.File(self.path[ifile], 'r') as f:
                data = f["fields"]['vorticity'][isim, it:it + self.n_input] # (n_in, channels, *datashape)
                target = f["fields"]['vorticity'][isim, it + self.n_input + self.n_stride:it + self.n_input + self.n_stride + self.n_output] # (n_out, channels, *datashape)
                
        return data, target

    def _load_data(self):
        self.data = []
        self.n_sim = 0
        if self.verbose: start_time = time.time()
        for path in self.path:
            with h5py.File(path, 'r') as f:
                _n_sim = f.attrs["n_trajectories"]
                
                end = min(self.max_n_sim - self.n_sim, _n_sim)
                vorticity = f["fields"]['vorticity'][:end]# vorticity = f["fields"]['vorticity'][:]
                if self.verbose: 
                    print(f'Data Loaded from{path}. shape: {vorticity.shape}, memory: {vorticity.nbytes/1e6} (MB)')
            
            self.data.append(vorticity)
            self.n_sim += len(vorticity)
            if self.max_n_sim <= self.n_sim: break
            
        self.data = np.concatenate(self.data, axis=0)
        memory = self.data.nbytes
        self.data = torch.from_numpy(self.data)
        # self.data = self.data.reshape(self.data.size(0) * self.data.size(1), self.data.size()[2:]) # self.data = torch.concat(self.data, dim=0)
        if self.verbose: print(f'Data Loaded. shape: {self.data.shape}, dtype: {self.data.dtype}, time: {time.time() - start_time} (sec), memory: {memory/1e6} (MB)')
        self._calc_len()
        
    def _build_metadata(self):
        
        with h5py.File(self.path[0], 'r') as f:
            self.nu = f['scalars']['nu'][()]

            self.t = t = f["dimensions"]["t"][:]
            self.x = x = f["dimensions"]["x"][:]
            self.y = y = f["dimensions"]["y"][:]
            self.X, self.Y = np.meshgrid(x, y)

            self.Lx = x[-1] - x[0]
            self.Ly = y[-1] - y[0]
            self.dx = x[1] - x[0]
            self.dy = y[1] - y[0]
            self.Nx = len(x)
            self.Ny = len(y)
            self.dt = t[1] - t[0]
            self.Nt = len(t)
            self.T = t[-1]
            self.t0 = t[0]

        self.n_sim_per_file = []
        for path in self.path:
            with h5py.File(path, 'r') as f:
                n_sim = f.attrs["n_trajectories"]
                self.n_sim_per_file.append(n_sim)
                if self.max_n_sim <= sum(self.n_sim_per_file): break
        
        self.metadata = {
            'nu': self.nu,

            'Lx': self.Lx,
            'Ly': self.Ly,
            'dx': self.dx,
            'dy': self.dy,
            'Nx': self.Nx, 
            'Ny': self.Ny,
            'x': self.x,
            'y': self.y,

            'dt': self.dt,
            'Nt': self.Nt,
            'T': self.T,
            't0': self.t0,
            't': self.t,
        }
        if self.verbose:
            print(f"Loaded dataset metadata: {self.metadata.keys()}")
        return self.metadata
    
    def _preprocess_data(self, data):
        data, target = data

        x = torch.cat([data, target], dim=0)

        return x
    def _postprocess_data(self, data):
        data, target = data[0], data[1]

        # data = data.flatten(0, 1) # data.reshape(data.shape[0] * data.shape[1], *data.shape[2:]) # (n_in * channels, *datashape)
        # target = target.flatten(0, 1) # target.reshape(target.shape[0] * target.shape[1], *target.shape[2:]) # (n_out * channels, *datashape)

        return data, target

class CustomDataModule:
    def __init__(self, 
                 batch_size:int=32, 
                 Ndata_train:int=500, 
                 Ndata_val:int=50, 
                 Ndata_test:int=100, 
                 num_workers:int=0,
                 transform:Optional[callable]=None,
                 normalization:Optional[callable]=None,
                 *args, **kwargs
                 ):
        self.batch_size = batch_size
        self.Ndata_train = Ndata_train
        self.Ndata_val = Ndata_val
        self.Ndata_test = Ndata_test

        self.num_workers = num_workers

        self.transform = transform
        self.normalization = normalization

        self.args = args
        self.kwargs = kwargs

    
    def setup(self, stage:str=None):
        if stage == 'fit' or stage is None:
            self.train_dataset = HIT2dDataset(split_name='train', max_n_sim=self.Ndata_train, transform=self.transform, normalization=self.normalization, *self.args, **self.kwargs, )
            self.val_dataset = HIT2dDataset(split_name='valid', max_n_sim=self.Ndata_val, transform=None, normalization=self.normalization, *self.args, **self.kwargs, )
            
            print(f'Train dataset: {len(self.train_dataset)}')
            print(f'Val dataset: {len(self.val_dataset)}')

        if stage == 'train':
            self.train_dataset = HIT2dDataset(split_name='train', max_n_sim=self.Ndata_train, transform=self.transform, normalization=self.normalization, *self.args, **self.kwargs, )
            print(f'Train dataset: {len(self.train_dataset)}')
            
        if stage == 'valid':
            self.val_dataset = HIT2dDataset(split_name='valid', max_n_sim=self.Ndata_val, transform=None, normalization=self.normalization, *self.args, **self.kwargs, )
            print(f'Val dataset: {len(self.val_dataset)}')

        if stage == 'test' or stage is None:
            self.test_dataset = HIT2dDataset(split_name='valid', max_n_sim=self.Ndata_test, transform=None, normalization=self.normalization, *self.args, **self.kwargs, )
            print(f'Test dataset: {len(self.test_dataset)}')
    
    def prepare_data(self):
        pass

    def train_dataloader(self):
        if self.train_dataset is None: 
            # assert False, "Error: train_dataset is None. run setup('fit') or setup('train') first."
            return None
        return torch.utils.data.DataLoader(self.train_dataset, 
                                           batch_size = self.batch_size, 
                                           shuffle=True, 
                                           num_workers=self.num_workers, 
                                           pin_memory=True,
                                           drop_last=True, 
                                           )
    
    def val_dataloader(self):
        if self.val_dataset is None: 
            # assert False, "Error: val_dataset is None. run setup('fit') or setup('valid') first."
            return None
        return torch.utils.data.DataLoader(self.val_dataset, 
                                           batch_size = self.batch_size, 
                                           shuffle=False, 
                                           num_workers=self.num_workers,
                                           pin_memory=True,
                                           drop_last=True,
                                           )

    def test_dataloader(self):
        if self.test_dataset is None: 
            # assert False, "Error: test_dataset is None. run setup('test') first."
            return None
        test_batch_size = self.batch_size # self.test_dataset.n_windows_per_sim # // 8
        return torch.utils.data.DataLoader(self.test_dataset, 
                                           batch_size = test_batch_size, 
                                           shuffle=False, 
                                           num_workers=self.num_workers, 
                                           pin_memory=True,
                                           drop_last=True,
                                           )


class RandomShift(nn.Module):
    def __init__(self, 
        shifts:Union[float, Sequence[float]]=(0.5, 0.5), 
        dims:Union[int, Sequence[int]]=(-2, -1),
        *args, **kwargs
        ):
        super(RandomShift, self).__init__()
        self.shifts = shifts
        self.dims = dims

    def forward(self, 
        x, 
        shifts:Union[float, Sequence[float]]=None, 
        dims:Union[int, Sequence[int]]=None, 
        ):
        if shifts is None: shifts = self.shifts
        if dims is None: dims = self.dims
        
        return self.RandomShift(x, shifts=self.shifts, dims=self.dims)
    
    def RandomShift(self,
        x, 
        shifts:Union[float, Sequence[float]]=(0.5, 0.5), 
        dims:Union[int, Sequence[int]]=(-2, -1)
        ):
        '''
        Random shift the input tensor along the spatial dimensions. 
        Parameters:
        - x (torch.tensor): input tensor, shape 
        - shifts (float or sequence of float): maximum shift fraction along each dimension. 
            If float, the same shift fraction is applied to all dimensions.
            If sequence of float, the length must be equal to the number of spatial dimensions.
            The shift fraction is relative to the size of the dimension.
        - dims (int or sequence of int): dimensions to apply the shift. 
            If int, the same dimension is applied to all spatial dimensions.
            If sequence of int, the length must be equal to the number of spatial dimensions.
        
        Returns:
        - x (torch.tensor): shifted tensor, shape 
        '''
        if isinstance(shifts, float): shifts = [shifts] * len(dims)
        assert len(shifts) == len(dims), "Length of shifts and dims must be equal to the number of spatial dimensions."

        shifts = [int(np.random.uniform(-s, s) * x.shape[d]) for s, d in zip(shifts, dims)]
        x = torch.roll(x, shifts=shifts, dims=dims)
        return x

class Reflection(nn.Module):
    def __init__(self, 
        p: float=0.5,
        dims:Union[int, Sequence[int]]=(-2, -1),
        *args, **kwargs
        ):
        super().__init__()
        self.p = p
        self.dims = dims

    def forward(self, 
        x, 
        p = None, 
        dims:Union[int, Sequence[int]]=None, 
        ):
        if dims is None: dims = self.dims
        if p is None: p = self.p
        
        return self.Reflection(x, p=p, dims=self.dims)
    
    def Reflection(self,
        x, 
        p: float=0.5, 
        dims:Union[int, Sequence[int]]=(-2, -1)
        ):
        '''
        Reflection the input tensor along the spatial dimensions. 
        Parameters:
        - x (torch.tensor): input tensor, shape 
        - dims (int or sequence of int): dimensions to apply the Reflection. 
            If int, the same dimension is applied to all spatial dimensions.
            If sequence of int, the length must be equal to the number of spatial dimensions.
        
        Returns:
        - x (torch.tensor): Reflection tensor, shape 
        '''
        if isinstance(dims, int): dims = [dims]
        
        for d in dims:
            if np.random.rand() < p: x = torch.flip(x, dims=(d,))
        return x

class Reverse(nn.Module):
    def __init__(self, 
        p: float=0.5,
        *args, **kwargs
        ):
        super().__init__()
        self.p = p

    def forward(self, 
        x, 
        p = None, 
        ):
        if p is None: p = self.p
        
        return self.Reverse(x, p=p)
    
    def Reverse(self,
        x, 
        p: float=0.5, 
        ):
        '''
        Reverse the input tensor along the time dimension. 
        Parameters:
        - x (torch.tensor): input tensor, shape 
        - dim (int): dimension to apply the Reverse. 
        
        Returns:
        - x (torch.tensor): Reverse tensor, shape 
        '''
        if np.random.rand() < p: x = -x 
        return x
    
class CustomTransform(nn.Module):
    def __init__(self, 
                *args, **kwargs, 
                ):
        super(CustomTransform, self).__init__()
        # self.file_dir = file_dir
        # self.Ndata = Ndata
        self.kwargs = kwargs 
        self.build_transform()

    def build_transform(self):
        self.transforms = nn.Sequential(
            RandomShift(shifts=0.5, dims=(-2, -1)),
            Reflection(p=0.5, dims=(-2, -1)),
            Reverse(p=0.5),
        )
        return self.transforms
    
    def forward(self, x):
        x = self.transform(x)
        return x
    
    def transform(self, x):
        for t in self.transforms:
            x = t(x)
        return x

    def inverse_transform(self, x):
        for t in self.transforms[::-1]:
            x = t.inverse_transform(x)
        return x

class Standardize(nn.Module):
    def __init__(self, 
        mean:Optional[float]=None, std:Optional[float]=None, 
        normalization_path:Optional[str]=None,

        base_path: Optional[str] = None,
        dataset_name: Optional[str] = None,
        split_name: Optional[str] = None,
        *args, **kwargs, 
        ):
        super(Standardize, self).__init__()
        self.args = args
        self.kwargs = kwargs

        ## build_transform
        if mean is not None and std is not None: 
            self.mean = mean
            self.std = std
        elif normalization_path is not None:
            self.load_stats(normalization_path)
        elif dataset_name is not None: 
            path = os.path.join(base_path, split_name, dataset_name)
            self.calc_stats_from_dataset(path)
        else:
            assert False, "Error: must provide either (mean, std), normalization_path, or (base_path, dataset_name, split_name) to calculate mean and std."
    
    def load_stats(self, path=None):
        assert os.path.exists(path), f"Error: normalization path {path} does not exist."
        with open(path, "r") as f:
            stats = yaml.safe_load(f)['statistics']

            self.mean = stats['mean']
            self.std = stats['std']
        return self.mean, self.std
    
    def calc_stats_from_dataset(self, path=None):
        dataset = HIT2dDataset(path=path, load_data=True, *self.args, **self.kwargs).load_data()
        self.mean = dataset.data.mean().item()
        self.std = dataset.data.std().item()
        return self.mean, self.std

    def forward(self, x):
        return self.normalize(x, params=None)
    
    def normalize(self, inpt: Any, params: Dict[str, Any]):
        return (inpt - self.mean) / self.std

    def denormalize(self, inpt: Any, params: Dict[str, Any]):
        return inpt * self.std + self.mean

In [7]:
class PltLogger:
    def __init__(self, 
                 fname: Optional[str]=None,
                 save_dir:Union[str, Path]='./', 
                 prefix:str='',
                 format:str='ascii', 
                 **kwargs,
                 ):
        self.fname = fname
        self.save_dir = save_dir 
        self.prefix=prefix 
        self.format = format
        self.kwargs = kwargs

    def log(self, 
            name:str, 
            value: Union[Tensor, int, float], 
            ):
        pass

    def log_dict(self, 
                 dictionary: Mapping[str, Union[Tensor, int, float]], 
                 ):
        # 이것도 만들기! log 출력할때 좋잖아 
        pass
    
    def save_2d_dict(self, 
                dictionary: Mapping[str, Union[Tensor, np.ndarray]],  
                time_step:Union[int, str]=0,
                isheader:bool=True,
                title:str=None,
                prefix:str=None, 
                save_dir:Union[str, Path]=None,
                fname:str=None, 
                format:str=None,
                fmt:str="%.6f",
                domain_mesh:list=None,
                domain_size:list=None, 
                ):
        pass
        
    def save_2d(self, 
                name:Union[list, str], 
                value: Union[torch.Tensor, np.ndarray], 
                time_step:Union[int, str]=0,
                isheader:bool=True,
                title:str=None,
                prefix:str=None, 
                save_dir:Union[str, Path]=None,
                fname:str=None, 
                format:str=None,
                fmt:str="%.6f",
                domain_mesh:list=None,
                domain_size:list=None, 
                ):
        if not format: format = self.format
        if not save_dir: save_dir = self.save_dir
        if not fname: fname = self.fname
        if not prefix: prefix = self.prefix
        if isinstance(value, torch.Tensor): value = value.detach().cpu().numpy()
        if isinstance(name, str): name = [name]
        for i in range(value.shape[0] - len(name)): name.append(f'var{i}')
        
        if value.ndim == 2: value.unsqueeze(0)
        if value.ndim != 3: 
            raise ValueError(f'Dimension Error {value.ndim-1}!=2')
            
        channels, nx, ny = value.shape 
        if domain_mesh is None:
            Lx, Ly = domain_size if domain_size else [nx+1, ny+1] # Domain size
            x = np.linspace(0., Lx, nx, endpoint=True)
            y = np.linspace(0., Ly, ny, endpoint=True)
            domain_mesh = np.meshgrid(x, y, indexing="ij")
        X, Y = domain_mesh

        if format == 'ascii':
            if isheader:
                with open(save_dir + fname + prefix + '.plt', 'w') as f:
                    if title: f.write(f'TITLE = "{title}"\n')
                    f.write("VARIABLES = x, y, " + ", ".join([f'"{na}"' for na in name]) + "\n")
            with open(save_dir + fname + prefix + '.plt', 'a') as f:
                f.write(f"ZONE T=\"T={time_step}\", I={nx}, J={ny}, DATAPACKING=POINT\n")
                data = np.column_stack([X.ravel(), Y.ravel(), value.reshape(channels, nx * ny).T])
                np.savetxt(f, data, fmt=fmt)

        elif format == 'binary':
            pass

    def save_1d_dict(self, 
                dictionary: Mapping[str, Union[Tensor, np.ndarray]], 
                time_step:Union[int, str]=0,
                iszone:bool=True,
                isheader:bool=True,
                title:str=None,
                prefix:str=None, 
                save_dir:Union[str, Path]=None,
                fname:str=None, 
                format:str=None,
                fmt:str="%.6f",
                ):
        if not format: format = self.format
        if not save_dir: save_dir = self.save_dir
        if not fname: fname = self.fname
        if not prefix: prefix = self.prefix
        dictionary = {key: value.detach().cpu().numpy() if isinstance(value, torch.Tensor) else value for key, value in dictionary.items()}

        if format == 'ascii':
            if isheader:
                with open(save_dir + fname + prefix + '.plt', 'w') as f:
                    if title: f.write(f'TITLE = "{title}"\n')
                    f.write("VARIABLES = " + ", ".join([f'"{na}"' for na in dictionary.keys()]) + "\n")
            with open(save_dir + fname + prefix + '.plt', 'a') as f:
                if iszone: f.write(f"ZONE T=\"T={time_step}\", I={len(next(iter(dictionary.values())))}, DATAPACKING=POINT\n")
                data = np.column_stack(dictionary.values())
                np.savetxt(f, data, fmt=fmt)
        elif format == 'binary':
            pass

    def save_1d(self, 
                name:Union[list, str], 
                value: Union[torch.Tensor, np.ndarray], 
                time_step:Union[int, str]=0,
                isheader:bool=True,
                title:str=None,
                prefix:str=None, 
                save_dir:Union[str, Path]=None,
                fname:str=None, 
                format:str=None,
                fmt:str="%.6f",
                ):
        if isinstance(name, str): name = [name]
        for i in range(value.shape[0] - len(name)): name.append(f'var{i}')
        dictionary = {na: val for na, val in zip(name, value)}
        self.save_1d_dict(dictionary, 
                     time_step=time_step, 
                     isheader=isheader, 
                     title=title, 
                     prefix=prefix, 
                     save_dir=save_dir, 
                     fname=fname, 
                     format=format, 
                     fmt=fmt
                     )


In [8]:
def spectral_relative_error(output:torch.Tensor, target:torch.Tensor, dim:int=2) -> torch.Tensor: 
    flag_dim = False
    if not dim: 
        dim = output.ndim - 1
        output = output.unsqueeze(0) 
        target = target.unsqueeze(0) 
        flag_dim = True
    batch_shape, channels, shape = output.shape[:-(dim+1)], output.shape[-(dim+1)], output.shape[-dim:] 
    fft_dim = tuple(range(-dim, 0))

    ## Compute the wavenumbers
    k = [torch.fft.fftfreq(n, 1/n, device=output.device) for n in shape] 
    k = torch.meshgrid(k, indexing='ij')
    k = torch.stack(k)
    k = torch.sqrt(torch.sum(k**2, axis=0)) 
    Nk = min(shape)//2

    ## Compute spectral relative error
    output_fft = torch.fft.fftn(output, dim=fft_dim, norm='forward')
    target_fft = torch.fft.fftn(target, dim=fft_dim, norm='forward')
    E_k = torch.sqrt((output_fft - target_fft)**2 / (target_fft**2 + 1e-8)).abs()

    ## Compute the spectrum
    spectrum = torch.zeros((*batch_shape, Nk + 1), device=output.device)
    for i in range(Nk + 1): spectrum[..., i] = (E_k[..., k == i]).mean(dim=fft_dim)
    
    spectrum = spectrum.mean(dim=0)
    if flag_dim: spectrum = spectrum.squeeze(0)
    return spectrum

def plot_histogram(tensor, title, xlabel, bins=50):
    bins = min(bins, len(tensor.flatten()))
    plt.hist(torch.abs(tensor).detach().cpu().numpy().flatten(), bins=bins, density=True, log=True)
    plt.title(title)
    plt.xlabel(xlabel)

def compute_spectrum(tensor, dim=2):
    batch_shape, channels, shape = tensor.shape[:-(dim+1)], tensor.shape[-(dim+1)], tensor.shape[-dim:]
    fft_dim = tuple(range(-dim, 0))

    k = [torch.fft.fftfreq(n, 1/n, device=tensor.device) for n in shape]
    k = torch.meshgrid(k, indexing='ij')
    k = torch.stack(k)
    k = torch.sqrt(torch.sum(k**2, axis=0)) 
    Nk = min(shape) // 2

    spectrum = torch.zeros((*batch_shape, Nk + 1), device=tensor.device)
    tensor = torch.fft.fftshift(tensor, dim=fft_dim)
    E_k = torch.abs(tensor)# .view(*batch_shape, -1)
    for i in range(Nk + 1): spectrum[..., i] = (E_k[..., k == i]).mean(dim=fft_dim)

    spectrum = spectrum.mean(dim=0)
    return spectrum

def plot_param_trends(log_history, param_type='params', 
                      title:str=None,
                fname:str=None, 
                xlabel:str='epoch',
                name:str='parameter trends',
                ):
    """
    log_history: list of logs collected over iterations
    param_type: 'params' or 'grad'
    """
    param_names = log_history[0][param_type].keys()
    x = np.arange(len(log_history))
    
    fig, ax = plt.subplots()
    for param in param_names:
        means = np.array([log[param_type][param]['mean'] for log in log_history])
        stds = np.array([log[param_type][param]['std'] for log in log_history])

        ax.plot(x, means, label=f'{param_type}_{param}')
        ax.fill_between(x, means - stds, means + stds, alpha=0.3)
    
    ax.set_xlabel(xlabel if xlabel else 'epoch')
    ax.set_ylabel('parameter value')
    ax.set_title(title if title else f'{name}')

    # Log the figure to wandb
    fig.savefig(f'./result/{fname}.png')
    # wandb.log({fname if title else f'{name}': wandb.Image(f'./result/{fname}.png')})
    plt.close(fig)

log_history = []
def hook(model: nn.Module):
    if not hasattr(model, 'iteration'): model.iteration = 1
    else: model.iteration += 1
    log = {}
    for name, param in model.named_parameters():
        weight = param.clone()
        # plot_histogram(torch.abs(weight), f'{name} Weight Distribution {weight.shape}', 'Weight')
        # plt.savefig('./result/'+f'{name}_WeightDistribution_{model.iteration}.png')
        # plt.close()
        wandb.log({f'{name}_WeightDistribution': wandb.Histogram(torch.abs(weight).detach().cpu().numpy())})
        # log['weight'] = {f'{name}': {'mean': weight.mean().item(), 'std': weight.std().item()}}

        if weight.is_complex(): 
            weight = rfft2fft(weight)
            weight_spectrum = compute_spectrum(weight)
            # plot_spectrum(weight_spectrum, title=f'{name} Weight Spectrum')
            # plt.savefig('./result/'+f'{name}_WeightSpectrum_{model.iteration}.png')
            # plt.close()
            # wandb.log({
            #     f'{name}_WeightSpectrum': wandb.plot.line(weight_spectrum.detach().cpu().numpy(), title=f'{name} Weight Spectrum'), 
            # })
            log_1d_dict_wandb(
                    dictionary={'spectrum': weight_spectrum.detach().cpu().numpy()}, 
                    title=f'{name}_WeightSpectrum',
                    fname=f'{name}_WeightSpectrum', 
                    xlabel='iteration', ylabel='value', 
                    xscale='log', yscale='log',
                    )

        if param.grad is not None: 
            grad = param.grad.clone()
            plot_histogram(torch.abs(grad), f'{name} Gradient Distribution {grad.shape}', 'Gradient')
            plt.savefig('./result/'+f'{name}_GradientDistribution.png')
            plt.close()
            wandb.log({f'{name}_GradientDistribution': wandb.Histogram(torch.abs(grad).detach().cpu().numpy())})
            

            if grad.is_complex(): 
                grad = rfft2fft(grad)
                grad_spectrum = compute_spectrum(grad)
                # plot_spectrum(grad_spectrum, title=f'{name} Gradient Spectrum')
                # plt.savefig('./result/'+f'{name}_GradientSpectrum_{model.iteration}.png')
                # plt.close()
                # wandb.log({
                # f'{name}_GradientSpectrum': wandb.plot.line(grad_spectrum.detach().cpu().numpy(), title=f'{name} Weight Spectrum'), 
                # })
                log_1d_dict_wandb(
                    dictionary={'spectrum': grad_spectrum}, 
                    title=f'{name}_GradientSpectrum',
                    fname=f'{name}_GradientSpectrum', 
                    xlabel='iteration', ylabel='value', 
                    xscale='log', yscale='log',
                    )
            # log['grad'] = {f'{name}': {'mean': grad.abs().mean().item(), 'std': grad.abs().std().item()}} if grad.is_complex() else {'mean': grad.mean().item(), 'std': grad.std().item()}

            grad_weight_ratio = torch.abs(grad) / (torch.abs(weight) + 1e-8)
            plot_histogram(grad_weight_ratio, f'{name} Gradient-Weight ratio Distribution {grad_weight_ratio.shape}', 'Gradient-Weight ratio' )
            plt.savefig('./result/'+f'{name}_RatioDistribution_{model.iteration}.png')
            plt.close()
            wandb.log({f'{name}_RatioDistribution': wandb.Histogram(torch.abs(grad_weight_ratio).detach().cpu().numpy())})

            if grad.is_complex(): 
                ratio_spectrum = compute_spectrum(grad_weight_ratio)
                # plot_spectrum(ratio_spectrum, title=f'{name} Gradient-Weight ratio Spectrum')
                # plt.savefig('./result/'+f'{name}_RatioSpectrum_{model.iteration}.png')
                # plt.close()
                # wandb.log({
                # f'{name}_RatioSpectrum': wandb.plot.line(ratio_spectrum.detach().cpu().numpy(), title=f'{name} Weight Spectrum'), 
                # })
                log_1d_dict_wandb(
                    dictionary={'spectrum': ratio_spectrum}, 
                    title=f'{name} Gradient-Weight ratio Spectrum',
                    fname=f'{name} Gradient-Weight ratio Spectrum', 
                    xlabel='iteration', ylabel='value', 
                    xscale='log', yscale='log',
                    )
                
        # wandb.log(log)
        
        # log_history.append(log)
        # if len(log_history) > 1:
        #     plot_param_trends(log_history, param_type='weight')
        #     plot_param_trends(log_history, param_type='grad')
def log_1d_colormap_wandb( 
                name:Union[list, str], 
                value: Union[torch.Tensor, np.ndarray], 
                title:str=None,
                fname:str=None, 
                xlabel:str='epoch',
                vmin:float=None, 
                vmax:float=None,
                ): 
    value = value.detach().cpu().numpy() if isinstance(value, torch.Tensor) else value
    if vmin is None: vmin = max(value.min(), 1e-8) 
    if vmax is None: vmax = value.max()

    fig, ax = plt.subplots()
    c = ax.imshow(
        value.T, 
        aspect='auto', 
        cmap='coolwarm', 
        # extent=[1, value.shape[0], 0.5, value.shape[1] + 0.5],
        origin='lower',
        norm = matplotlib.colors.LogNorm(vmin=vmin, vmax=vmax),
    )
    
    ax.set_xlabel(xlabel if xlabel else 'epoch')
    ax.set_ylabel('wavenumber')
    ax.set_title(title if title else f'{name} Colormap')
    fig.colorbar(c, ax=ax)

    # Log the figure to wandb
    fig.savefig(f'./result/{fname}.png')
    wandb.log({fname if title else f'{name} Colormap': wandb.Image(f'./result/{fname}.png')})
    plt.close(fig)

def log_1d_dict_wandb(
                dictionary: Mapping[str, Union[Tensor, np.ndarray]], 
                title:str=None,
                fname:str=None, 
                xlabel:str='epoch', ylabel:str='value', 
                xscale:str='linear', yscale:str='linear',
                ): 
    if isinstance(dictionary, dict): dictionary = {k: v.detach().cpu().numpy() if isinstance(v, torch.Tensor) else v for k, v in dictionary.items()}
    fig, ax = plt.subplots()
    for key, value in dictionary.items(): ax.plot(value, label=key)
    ax.set(xscale=xscale, yscale=yscale, 
           xlabel=xlabel, ylabel=ylabel, 
           title=title if title else f'{fname}',
           )
    if xlabel == 'log' and ylabel == 'log': 
        # ax.set_xticks(xticks = [2 ** (np.round(np.log2(Nk)).astype(np.int32) - i) for i in range(min(np.round(np.log2(Nk)).astype(np.int32), 7))],)
        ax.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
    if len(dictionary) > 1: ax.legend()

    # Log the figure to wandb
    fig.savefig(f'./result/{fname}.png')
    wandb.log({fname if title else f'{fname}': wandb.Image(f'./result/{fname}.png')})
    plt.close(fig)

def rfft2fft(x: torch.Tensor) -> torch.Tensor:
    """
    Convert a complex-valued rfft tensor to a complex-valued fft tensor.
    
    Args:
        x (torch.Tensor): A complex-valued rfft tensor of shape (batch, channel, *data_shape).
        dim (int): Number of spatial dimensions.
    
    Returns:
        torch.Tensor: A complex-valued fft tensor with the same shape as x.
    """
    if (x.shape[-1] - 1) * 2 % 2 == 0: 
        # 짝수인 경우: DC 성분만 고유하므로 제외하고 반전 후 켤레 취함
        conj_part = torch.conj(x[..., 1:-1].flip(-1))
    else: 
        # 홀수인 경우: DC와 Nyquist 성분은 고유하므로 rfft_tensor의 [1:-1] 구간만 반전 후 켤레 취함 
        conj_part = torch.conj(x[..., 1:].flip(-1)) # x = torch.cat([x, torch.flip(x[..., 1:-1], dims=fft_dim)], dim=-1)
    return torch.cat([x, conj_part], dim=-1)

def fft2rfft(x: torch.Tensor) -> torch.Tensor:
    """
    Convert a complex-valued fft tensor to a complex-valued rfft tensor.
    
    Args:
        x (torch.Tensor): A complex-valued fft tensor of shape (batch, channel, *data_shape).
        dim (int): Number of spatial dimensions.
    
    Returns:
        torch.Tensor: A complex-valued rfft tensor with the same shape as x.
    """
    rfft_last_dim = x.shape[-1] // 2 + 1

    return x[..., :rfft_last_dim]

def enforce_conjugate_symmetry(x: torch.Tensor, isrfft:bool=False, isshift:bool=False) -> torch.Tensor:
    """
    Enforces conjugate symmetry on a 2D complex tensor in Fourier space.
    The input tensor x is expected to have shape:
        (batch_size, channels, n_x, n_y)
        
    For each element, this function sets:
        x[..., i, j] = 0.5 * (x[..., i, j] + conj(x[..., i_sym, j_sym]))
    where (i_sym, j_sym) are the symmetric indices computed as:
        i_sym = (-i) % n_x,  j_sym = (-j) % n_y

    Args:
        x (torch.Tensor): Input complex-valued tensor with shape (B, C, n_x, n_y)
    
    Returns:
        torch.Tensor: The tensor with enforced conjugate symmetry.
    """
    if not isinstance(x, torch.Tensor): x = x.to_tensor()
    
    if isrfft: x = rfft2fft(x)
    # if isshift: x = torch.fft.fftshift(x, dim=(-2, -1))
    # Get spatial dimensions.
    n_x, n_y = x.shape[-2:]
    # Create index grids for the spatial dimensions.
    ii, jj = torch.meshgrid(torch.arange(n_x, device=x.device),
                             torch.arange(n_y, device=x.device),
                             indexing='ij')
    # Compute symmetric indices.
    ii_sym = (-ii) % n_x
    jj_sym = (-jj) % n_y

    # Compute the symmetric average for every spatial element
    # This operation broadcasts over the batch and channel dimensions.
    x = 0.5 * (x[..., ii, jj] + torch.conj(x[..., ii_sym, jj_sym]))
    # if isshift: x = torch.fft.ifftshift(x, dim=(-2, -1))
    if isrfft: x = fft2rfft(x)
    
    return x

def check_conjugate_symmetry(x: torch.Tensor, isrfft:bool=False, isshift:bool=False) -> float:
    """
    Checks whether a 2D complex tensor in Fourier space satisfies
    the conjugate symmetry condition for real-valued physical data:
        x(-k) == conj(x(k))
    The tensor is expected to have shape:
        (batch_size, channels, n_x, n_y)
        
    Args:
        x (torch.Tensor): Complex-valued tensor with shape (B, C, n_x, n_y)
        tol (float): Tolerance used for comparing floating-point values.
    
    Returns:
        bool: True if the tensor satisfies conjugate symmetry within the given tolerance,
              otherwise False.
    """
    if not isinstance(x, torch.Tensor): x = x.to_tensor()
    # if isshift: x = torch.fft.fftshift(x, dim=(-2, -1))
    if isrfft: x = rfft2fft(x)
    # Get spatial dimensions.
    n_x, n_y = x.shape[-2:]
    # Create index grids.
    ii, jj = torch.meshgrid(torch.arange(n_x, device=x.device),
                             torch.arange(n_y, device=x.device),
                             indexing='ij')
    # Compute symmetric indices.
    ii_sym = (-ii) % n_x
    jj_sym = (-jj) % n_y

    # Compute the absolute difference between each element and its conjugate symmetric
    diff = torch.abs(x[..., ii, jj] - torch.conj(x[..., ii_sym, jj_sym]))
    # Check if all differences are within the tolerance.
    # return torch.all(diff < tol).item()
    return diff.mean()

In [ ]:
import torch
import torch.nn as nn

NORM_LAYERS = {
    'none': lambda out_channels: nn.Identity(),
    'batchnorm': lambda out_channels: nn.BatchNorm2d(out_channels),
    'instancenorm': lambda out_channels: nn.InstanceNorm2d(out_channels),
}

ACTIVATIONS = {
    'LeakyReLU': lambda: nn.LeakyReLU(0.2),
    'GELU': lambda: nn.GELU(),
    'ReLU': lambda: nn.ReLU(),
}

class DoubleConv(nn.Module):
    """(Conv → Act) × 2, padding=1 to keep H×W."""
    def __init__(self, in_channel:int, out_channel:int, 
                 activation=nn.ReLU(),
                 norm=nn.Identity(),
                 ):
        super().__init__()
        
        if isinstance(activation, str):
            activation = ACTIVATIONS[activation]()
        if isinstance(norm, str):
            norm = NORM_LAYERS[norm](out_channel)

        self.net = nn.Sequential(
            nn.Conv2d(in_channel, out_channel, kernel_size=3, padding=1, padding_mode='circular'),
            norm, 
            activation,
            nn.Conv2d(out_channel, out_channel, kernel_size=3, padding=1, padding_mode='circular'),
            norm,
            activation
        )
    def forward(self, x):
        return self.net(x)

class Down(nn.Module):
    """Pool → DoubleConv."""
    def __init__(self, in_channel:int, out_channel:int, 
                 activation=nn.ReLU(), 
                 pooling:callable=nn.MaxPool2d(kernel_size=2, stride=2), 
                 norm=nn.Identity(),
                 ):
        super().__init__()
        if isinstance(activation, str):
            activation = ACTIVATIONS[activation]()
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2) # pooling
        self.conv = DoubleConv(in_channel, out_channel, activation=activation, norm=norm)
        
    def forward(self, x):
        x = self.pool(x)  # [B, in_channel, H/2, W/2]
        x = self.conv(x)  # [B, out_channel, H/2, W/2]
        return x

class Up(nn.Module):
    """TransposedConv(in_channel→out_channel) → concat → DoubleConv(2*out_channel→out_channel)."""
    def __init__(self, in_channel:int, out_channel:int, 
                 activation=nn.ReLU(), 
                 norm=nn.Identity(),
                 ):
        super().__init__()
        
        if isinstance(activation, str):
            activation = ACTIVATIONS[activation]()
        if isinstance(norm, str):
            norm = NORM_LAYERS[norm](out_channel)
        self.norm = norm
        # in_channel: e.g. 1024 → out_channel: 512
        self.up   = nn.ConvTranspose2d(in_channel, out_channel, kernel_size=2, stride=2)
        self.conv = DoubleConv(2*out_channel, out_channel, activation=activation, norm=norm)
        
        

    def forward(self, x_dec, x_enc):
        x = self.up(x_dec)      # [B, out_channel, H*2, W*2]
        x = self.norm(x)
        # no crop needed since padding=1 kept encoder dims the same
        x = torch.cat([x_enc, x], dim=1)  # [B, 2*out_channel, H*2, W*2]
        x = self.conv(x)     # [B, out_channel, H*2, W*2]
        return x

class UNet(nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        activation = nn.ReLU(),
        pooling: callable = nn.MaxPool2d(kernel_size=2, stride=2), 
        **kwargs,
        ):
        super().__init__()
        self.kwargs = kwargs

        channels_ratio = kwargs.get('embedding_channels', 1024)/1024
        norm = kwargs.get('norm', 'none')
        
        
        # encoder
        self.inc   = DoubleConv(in_channels,  
                                int(64*channels_ratio),  
                                activation=activation, norm=norm)
        self.down1 = Down(int(64*channels_ratio),   
                          int(128*channels_ratio), 
                          activation=activation, pooling=pooling, norm=norm)
        self.down2 = Down(int(128*channels_ratio),  
                          int(256*channels_ratio), 
                          activation=activation, pooling=pooling, norm=norm)
        self.down3 = Down(int(256*channels_ratio),  
                          int(512*channels_ratio), 
                          activation=activation, pooling=pooling, norm=norm)
        self.down4 = Down(int(512*channels_ratio), 
                          int(1024*channels_ratio), 
                          activation=activation, pooling=pooling, norm=norm)
        # decoder
        self.up1   = Up(int(1024*channels_ratio), 
                        int(512*channels_ratio), 
                        activation=activation, norm=norm)
        self.up2   = Up(int(512*channels_ratio),  
                        int(256*channels_ratio), 
                        activation=activation, norm=norm)
        self.up3   = Up(int(256*channels_ratio),  
                        int(128*channels_ratio), 
                        activation=activation, norm=norm)
        self.up4   = Up(int(128*channels_ratio),   
                        int(64*channels_ratio), 
                        activation=activation, norm=norm)
        # final 1×1 conv
        self.outc  = nn.Conv2d(int(64*channels_ratio), out_channels, kernel_size=1)

    def forward(self, x):
        x1 = self.inc(x)      # [B,64, H,  W]
        x2 = self.down1(x1)   # [B,128,H/2,W/2]
        x3 = self.down2(x2)   # [B,256,H/4,W/4]
        x4 = self.down3(x3)   # [B,512,H/8,W/8]
        x5 = self.down4(x4)   # [B,1024,H/16,W/16]
        x  = self.up1(x5, x4) # [B,512,H/8,W/8]
        x  = self.up2(x,  x3) # [B,256,H/4,W/4]
        x  = self.up3(x,  x2) # [B,128,H/2,W/2]
        x  = self.up4(x,  x1) # [B,64, H,  W]
        return self.outc(x)   # [B,out_channels,H,W]



In [ ]:
class CustomModule(nn.Module):
    def __init__(self, 
                 modelconfig, 
                 optconfig, 
                 **kwargs, 
                 ):
        super().__init__()
        
        self.modelconfig = modelconfig
        self.optconfig = optconfig
        self.hparams = kwargs
        self.device = device

        self.model = None
        self.build_model()
        
        self.optimizer = None
        self.scheduler = None
        self.configure_optimizers()

        self.loss_fn = test_criterion[optconfig['loss_fn']]

    def build_model(self):
        self.model = UNet(
            **self.modelconfig['params'], 
            ).to(self.device)
        # self.model = neuralop.models.FNO(**self.modelconfig['params'], ).to(self.device)

        return self.model
    
    def configure_optimizers(self):
        if self.optimizer is not None: 
            return self.optimizer, self.scheduler
        self.optimizer = torch.optim.Adam(
            self.model.parameters(), 
            **self.optconfig['params']
        )

        self.scheduler = []
        SCHEDULERS = {
            'StepLR': torch.optim.lr_scheduler.StepLR, 
            'ExponentialLR': torch.optim.lr_scheduler.ExponentialLR, 
        }
        if self.optconfig['scheduler']: 
            SCHEDULERS[self.optconfig['scheduler']['name']](
                self.optimizer, 
                **self.optconfig['scheduler']['params'],
                ) 
        return self.optimizer, self.scheduler
    
    def forward(self, data):
        return self.model(data.to(self.device))
    
    def training_step(self, batch, batch_idx, ret_log:bool=False):
        data, target = batch

        # data = self.high2low(data, scale_factor=0.5, keepdim=True)
        # target = self.high2low(target, scale_factor=0.5, keepdim=True)
        self.optimizer.zero_grad()
        output = self.forward(data.to(self.device))
        loss = self.loss_fn(output, target.to(self.device))
        loss.backward()

        # if batch_idx % 3000 == 0:
        #     if not hasattr(self, 'error'): self.error = []
            
        #     # hook(self.model)
            
        #     self.error.append(spectral_relative_error(output, target.to(self.device), dim=2)[1:].detach().cpu().numpy())
        #     log_1d_colormap_wandb( 
        #         name='wavenumber', 
        #         value=np.stack(self.error), 
        #         title='spectral_relative_error',
        #         fname='spectral_relative_error', 
        #         xlabel='iteration',
        #         vmin=5e-2, vmax=2e1,
        #         )
            
        self.optimizer.step()
        
        self.log('loss', loss.item(), on_step=True)
        # log = {}
        # log.update({"loss": loss.item(), "output": output.detach().cpu(), 'batch': [data, target.detach().cpu()], # batch, 
        #        'learning_rate': [], })
        # for i in range(len(self.scheduler)): log['learning_rate'].append(self.scheduler[i].get_last_lr())
        # return log
        return loss.item()

    def validation_step(self, batch, batch_idx, ret_log:bool=True):
        # self.model.eval()
        data, target = batch

        # data = self.high2low(data, scale_factor=0.5, keepdim=True)
        # target = self.high2low(target, scale_factor=0.5, keepdim=True)

        output = self.forward(data.to(self.device))
        loss = self.loss_fn(output, target.to(self.device))
        self.log('val_loss', loss.item(), on_step=False)
        if ret_log: 
            log = {}
            log.update({"loss": loss.item(), "output": output.detach().cpu(), 'batch': [data, target.detach().cpu()]})
            return log
        return output

    def test_step(self, batch, batch_idx, ret_log:bool=False):
        # self.model.eval()
        data, target = batch

        output = self.forward(data.to(self.device))
        loss = self.loss_fn(output, target.to(self.device))

        if ret_log: output = {"loss": loss.item(), "output": output.detach().cpu()}
        return output
    
    def save(self, path, checkpoint:dict={}):
        checkpoint['model_state_dict'] = self.model.state_dict()
        checkpoint['optimizer_state_dict'] = self.optimizer.state_dict()
        checkpoint['scheduler'] = []
        for i in range(len(self.scheduler)):checkpoint[f'scheduler'].append(self.scheduler[i].state_dict())
        torch.save(checkpoint, path + '.pth')

    def load(self,
        path, 
        verbose=True, 
        load_opt:bool=True,
        freeze:bool=False,
        ):
        if os.path.exists(path + '.pth') and os.path.getsize(path + '.pth'):
            checkpoint = torch.load(path + '.pth', weights_only=False)

            self.model.load_state_dict(
                checkpoint['model_state_dict'], 
                strict=False,
                )
            if freeze: 
                for name, param in self.model.named_parameters():
                    if name in checkpoint['model_state_dict']:
                        param.requires_grad = False
            if verbose: print(f'model loaded from {path}')
            if load_opt:
                if self.optimizer and 'optimizer_state_dict' in checkpoint: 
                    self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                    if verbose: print(f'optimizer loaded from {path}')
                if self.scheduler and 'scheduler' in checkpoint: 
                    for i in len(self.scheduler): self.scheduler[i].load_state_dict(checkpoint['scheduler'][i])
                    if verbose: print(f'scheduler loaded from {path}')
    
    def log(self, name, value, prog_bar=False, logger=None, on_step=None, on_epoch=None, reduce_fx='mean', enable_graph=False, sync_dist=False, sync_dist_group=None, add_dataloader_idx=True, batch_size=None, metric_attribute=None, rank_zero_only=False):
        commit = False if on_step is False else True
        wandb.log({name: value}, commit=commit)

In [11]:
class BaseCallback:
    def __init__(self):
        pass
    
    def setup(self, trainer, pl_module, stage): pass
    def state_dict(self): pass

    def load_state_dict(self, state_dict): pass
    def on_load_checkpoint(self, trainer, pl_module, checkpoint): pass
    def on_save_checkpoint(self, trainer, pl_module, checkpoint): pass
    
    def on_fit_start(self, trainer, pl_module): pass
    def on_fit_end(self, trainer, pl_module): pass

    def on_train_batch_start(self, trainer, pl_module, batch, batch_idx): pass
    def on_train_batch_end(self, trainer, pl_module, outputs, batch, batch_idx): pass
    def on_train_epoch_start(self, trainer, pl_module): pass
    def on_train_epoch_end(self, trainer, pl_module): pass
    def on_train_start(self, trainer, pl_module): pass
    def on_train_end(self, trainer, pl_module): pass

    def on_validation_batch_start(self, trainer, pl_module, batch, batch_idx): pass
    def on_validation_batch_end(self, trainer, pl_module, outputs, batch, batch_idx): pass
    def on_validation_epoch_start(self, trainer, pl_module): pass
    def on_validation_epoch_end(self, trainer, pl_module): pass
    def on_validation_start(self, trainer, pl_module): pass
    def on_validation_end(self, trainer, pl_module): pass

    def on_predict_batch_start(self, trainer, pl_module, batch, batch_idx, dataloader_idx=0): pass
    def on_predict_batch_end(self, trainer, pl_module, batch, batch_idx, dataloader_idx=0): pass
    def on_predict_epoch_start(self, trainer, pl_module): pass
    def on_predict_epoch_end(self, trainer, pl_module): pass
    def on_predict_start(self, trainer, pl_module): pass
    def on_predict_end(self, trainer, pl_module): pass

    def on_test_batch_start(self, trainer, pl_module, batch, batch_idx, dataloader_idx=0): pass
    def on_test_batch_end(self, trainer, pl_module, batch, batch_idx, dataloader_idx=0): pass
    def on_test_epoch_start(self, trainer, pl_module): pass
    def on_test_epoch_end(self, trainer, pl_module): pass
    def on_test_start(self, trainer, pl_module): pass
    def on_test_end(self, trainer, pl_module): pass

    def on_after_backward(self, trainer, pl_module): pass
    def on_before_backward(self, trainer, pl_module, loss): pass
    def on_before_optimizer_step(self, trainer, pl_module, optimizer): pass
    def on_before_zero_grad(self, trainer, pl_module, optimizer): pass

class CustomCallback(BaseCallback):
    def __init__(self, 
                 criterions, 
                 file_dir:str='', 
                 fname:str ='',
                 device:str =None, 
                 val_every_n_iter:int=None,
                 ):
        super(CustomCallback, self).__init__()
        self.criterions = criterions # DL criterion + CFD criterion + DL part-of-loss
        self.file_dir = file_dir
        self.fname = fname
        self.device=device
        
    def on_validation_epoch_start(self, trainer, pl_module):
        self.time_stamp = time.time()
        trainer._current_val_return = {'loss': [], 'time': []}
        for key in self.criterions.keys():
            trainer._current_val_return[key] = []

    def on_validation_batch_end(self, trainer, pl_module, outputs, batch: Any, batch_idx: int, dataloader_idx: int = 0) -> None:
        data, target = outputs['batch'] # batch
        # trainer._current_val_return['batch'] = outputs['batch'] # batch
        # trainer._current_val_return['output'] = outputs['output']
        # trainer._current_val_return['loss'].append(outputs['loss'])

        for key in self.criterions.keys():
            value = self.criterions[key](outputs['output'].to(self.device), target.to(self.device)).item()

            trainer._current_val_return[key].append(value)
    
    def on_validation_end(self, trainer, pl_module, fname='') -> None:
        fname = self.fname + fname
        # data, target = trainer._current_val_return['batch']
        # output = trainer._current_val_return['output']
        # if trainer.logger: trainer.logger.log({'val_loss': np.mean(trainer._current_val_return['loss'])})

        epoch = trainer.current_epoch
        print(f'  Validation Epoch: {epoch}', end='')
        for key in self.criterions.keys():
            value = np.mean(trainer._current_val_return[key])
            print(f", {key}: {value:.6f}", end='')

            if trainer.logger: trainer.logger.log({'Val/epoch/'+key: value})
        print()

        plt.close('all')

        # ## enstrophy spectrum
        # spectrum1 = calc_energy_spectrum(target[:,0,:], dim=2) # np.mean([calc_enstrophy_spectrum(target[i,0,:]) for i in range(len(target))], axis=0); 
        # spectrum2 = calc_energy_spectrum(output[:,0,:], dim=2) # np.mean([calc_enstrophy_spectrum(output[i,0,:]) for i in range(len(output))], axis=0)
        # ## vorticity pdf
        # bin1, pdf1 = calc_pdf(target[:,0,:], dim=2) # np.mean([calc_pdf(target[i,0,:]) for i in range(len(target))], axis=0); 
        # bin2, pdf2 = calc_pdf(output[:,0,:], dim=2) # np.mean([calc_pdf(output[i,0,:]) for i in range(len(output))], axis=0)

        # # ### solution field 
        # plot_2d_surface(target[0,0,...,0], output[0,0,...,0], # axes=[ax1,ax2], 
        #                 kwargs={'suptitle': f'epoch={epoch}', 'figsize': (8, 4)}
        #                 )
        # plt.savefig(self.file_dir + fname + f'_Val_epoch={epoch}_field.png')
        # # plt.show()
        # plt.close()

        # fig = plt.figure(figsize=(8,4))
        # fig.suptitle(f'epoch={epoch}')
        # ax = fig.add_subplot(1, 2, 1)
        # plot_spectrum(spectrum1, spectrum2, ax=ax, kwargs={'title': f'enstrophy spectrum'})

        # ax = fig.add_subplot(1, 2, 2)
        # plot_pdf(bin1, pdf1, bin2, pdf2, ax=ax, kwargs={'title': f'pdf'})
        # plt.tight_layout()
        # plt.savefig(self.file_dir + fname + f'_Val_epoch={epoch}_stat.png')
        # # plt.show()
        # plt.close()

        ## frequency solution field 
        # self.n_modes = [[0, 8], [8, 16], [16, 32], ]
        # data_freq = self.preprocessing(data)
        # target_freq = self.preprocessing(target)
        # output_freq = self.preprocessing(output)
        # for i in range(len(target_freq)):
        #     _data = data_freq[i][0,0].detach().cpu().numpy()
        #     _target = target_freq[i][0,0].detach().cpu().numpy()
        #     _output = output_freq[i][0,0].detach().cpu().numpy()

        #     plot_2d_surface(
        #         u1=_data, 
        #         u2=[_target, _output],  
        #         cm1=True, cm2=True, vmax=None, vmin=None, 
        #         suptitle=f'epoch={epoch}',
        #         title1=f'input',
        #         title2=f'target',
        #         title3=f'prediction',
        #     )
        #     plt.savefig(self.file_dir + fname + f'_Val_epoch={epoch}_freq={i}_stat.png')
        #     plt.close()

        #     key = f'loss_{i}'
        #     value = ((_target - _output)**2).mean()
        #     print(f", {key}: {value:.6f}", end='')
        #     if trainer.logger: 
        #         trainer.logger.log({'Val/epoch/'+key: value})

        #         image = np.concatenate((_target, _output), axis=1)
        #         trainer.logger.log({'Val/field': wandb.Image(image)})

        ## save .wandb
        # image = np.concatenate((target[0,0,...,0], output[0,0,...,0]), axis=1)
        # if trainer.logger: trainer.logger.log({'Val/field': wandb.Image(image)})
        # if trainer.logger: trainer.logger.log({'Val/spectrum': wandb.plot.line_series(
        #     xs = [np.arange(0, spectrum1.shape[-1]+1), np.arange(0, spectrum2.shape[-1]+1)], 
        #     ys = [spectrum1, spectrum2],
        #     keys = ['ground truth', 'prediction'],
        #     xname = ['k', 'enstrophy spectrum'],
        #     title='enstrophy_spectrum', 
        #     )})
        # if trainer.logger: trainer.logger.log({'Val/pdf': wandb.plot.line_series(
        #     xs=[bin1, bin2], 
        #     ys = [pdf1, pdf2],
        #     keys = ['ground truth', 'prediction'],
        #     xname = ['w', 'pdf'],
        #     title='pdf', 
        #     )})

        # ## save .plt 
        # w1 = target[0,0,...,0]; w2 = output[0, 0, :]
        # Nx, Ny = target[0,0,...,0].shape
        # x = np.linspace(0., 2.*np.pi, Nx, endpoint=True)
        # y = np.linspace(0., 2.*np.pi, Ny, endpoint=True)
        # with open(self.file_dir + self.fname + '_Val_field' + '.plt', 'a') as f:
        #     f.write(f'Zone T="T={epoch}" I={Nx} J={Ny}\n')
        #     for ix in range(Nx):
        #         for iy in range(Ny):
        #             f.write(f'{x[ix]:.10f} {y[iy]:.10f} {w1[ix, iy]:.10f} {w2[ix, iy]:.10f}\n')

        # 
        # with open(self.file_dir + self.fname + '_Val_spectrum' + '.plt', 'a') as f:
        #     f.write(f'Zone T="T={epoch}" I={Nk}\n')
        #     for ik in range(Nk):
        #         f.write(f'{k_1d[ik]} {spectrum1[ik]:.10f} {spectrum2[ik]:.10f}\n')

        # n = bin1.shape[-1]
        # with open(self.file_dir + self.fname + '_pdf' + '.plt', 'a') as f:
        #     f.write(f'Zone T="T={epoch}" I={n}\n')
        #     for i in range(n):
        #         f.write(f'{bin1[i]:.10f} {pdf1[i]:.10f} {bin2[i]:.10f} {pdf2[i]:.10f}\n')


    def on_test_batch_start(self, trainer, pl_module, batch: Any, batch_idx: int, dataloader_idx: int = 0) -> None:
        self.time_stamp = time.time()

    def on_test_batch_end(self, trainer, pl_module, outputs, batch: Any, batch_idx: int, dataloader_idx: int = 0) -> None:
        data, target = batch
        trainer._current_test_return['batch'] = batch
        output = trainer._current_test_return['output'] = outputs['output']
        
        runtime = time.time() - self.time_stamp
        if trainer.logger: trainer.logger.log({'Test/time': runtime})
        print(f'Test sample {batch_idx}(time: {runtime:.6f}, batch size: {len(output)})', end='')
        for key in self.criterions.keys():
            value = self.criterions[key](outputs['output'].to(self.device), target.to(self.device)).item()

            trainer._current_test_return[key].append(value)
            print(f", {key}: {value:.6f}", end='')
        print()
        
        ## enstrophy spectrum
        spectrum1 = calc_energy_spectrum(target[:,0,:], dim=2) # np.mean([calc_enstrophy_spectrum(target[i,0,:]) for i in range(len(target))], axis=0); 
        spectrum2 = calc_energy_spectrum(output[:,0,:], dim=2) # np.mean([calc_enstrophy_spectrum(output[i,0,:]) for i in range(len(output))], axis=0)
        ## vorticity pdf
        bin1, pdf1 = calc_pdf(target[:,0,:], dim=2) # np.mean([calc_pdf(target[i,0,:]) for i in range(len(target))], axis=0); 
        bin2, pdf2 = calc_pdf(output[:,0,:], dim=2) # np.mean([calc_pdf(output[i,0,:]) for i in range(len(output))], axis=0)

        # ### solution field 
        plot_2d_surface(target[0,0,...,0], output[0,0,...,0], # axes=[ax1,ax2], 
                        kwargs={'figsize': (8, 4)}
                        )
        plt.savefig(self.file_dir + self.fname + f'_Test_sample={batch_idx}_field.png')
        # plt.show()
        plt.close()

        fig = plt.figure(figsize=(8,4))
        ax = fig.add_subplot(1, 2, 1)
        plot_spectrum(spectrum1, spectrum2, ax=ax, kwargs={'title': f'enstrophy spectrum'})

        ax = fig.add_subplot(1, 2, 2)
        plot_pdf(bin1, pdf1, bin2, pdf2, ax=ax, kwargs={'title': f'pdf'})
        plt.tight_layout()
        plt.savefig(self.file_dir + self.fname + f'_Test_sample={batch_idx}_stat.png')
        # plt.show()
        plt.close()

        ## save .wandb
        Nt = len(output)
        for it in range(Nt):
            image = np.concatenate((target[it,0,:], output[it,0,:]), axis=1)
            if trainer.logger: trainer.logger.log({'Test/'+f'sample_{batch_idx}/'+'field': wandb.Image(image)})
        if trainer.logger: trainer.logger.log({'Test/spectrum': wandb.plot.line_series(
            xs=[np.arange(0, spectrum1.shape[-1]+1), np.arange(0, spectrum2.shape[-1]+1)], 
            ys = [spectrum1, spectrum2],
            keys = ['ground truth', 'prediction'],
            xname = ['k', 'enstrophy spectrum'],
            title='enstrophy_spectrum', 
            )})
        if trainer.logger: trainer.logger.log({'Test/pdf': wandb.plot.line_series(
            xs=[bin1, bin2], 
            ys = [pdf1, pdf2],
            keys = ['ground truth', 'prediction'],
            xname = ['w', 'pdf'],
            title='pdf', 
            )})

    def on_test_end(self, trainer, pl_module) -> None:
        data, target = trainer._current_test_return['batch']
        output = trainer._current_test_return['output']

        print('Test ')
        for key in self.criterions.keys():
            value = np.mean(trainer._current_test_return[key])
            print(f", {key}: {value:.6f}", end='')

            if trainer.logger: trainer.logger.log({'Test/'+key: value})
        print()

    def preprocessing(self, x):
        n_modes = self.n_modes
        x = self.split(x, modes=n_modes)
        # x = self.scaling(x, dim=2, isNormalize=False)
        return x 
    
    def postprocessing(self, x):
        n_modes = self.n_modes
        # x = self.unscaling(x)
        x = self.unsplit(x, modes=n_modes)
        return x
    
    def scaling(self, x, 
                dim=1, 
                isNormalize:bool=False):
        dim = list(range(-dim, 0))
        if isNormalize:
            self.mean = x.amin(dim=dim, keepdim=True)
            self.std = x.amax(dim=dim, keepdim=True) - self.mean
        else: 
            self.mean = x.mean(dim=dim, keepdim=True)
            self.std = x.std(dim=dim, keepdim=True)
        return (x - self.mean) / self.std
    
    def unscaling(self, x):
        return x * self.std + self.mean
    
    def split(self, x, 
              modes=None
              ):

        b, c, *data_shape = x.shape
        fft_dims = list(range(-len(data_shape), 0))
        Fx = torch.fft.fft2(x, dim=fft_dims, norm='forward')
        Fx_ = torch.zeros((len(modes), b, c, *data_shape), dtype=Fx.dtype, device=x.device)
        
        k = [torch.fft.fftfreq(n, d=1./n) for n in data_shape]
        # k += [torch.fft.rfftfreq(data_shape[-1], d=1./data_shape[-1])] if Fx.dtype in [torch.float16, torch.float32, torch.float64] else [torch.fft.fftfreq(data_shape[-1], d=1./data_shape[-1])]


        k = torch.meshgrid(k, indexing='ij')
        k = torch.stack(k)
        k = torch.sqrt(torch.sum(k**2, axis=0)).to(x.device)
        for i in range(len(modes)):
            k1, k2 = modes[i]
            idx = (k1 <= k) & (k <= k2)
            Fx_[i] = Fx * idx

        x_ = torch.fft.ifft2(Fx_, dim=fft_dims, norm='forward').real # x_ = Fx_ # # 
        
        return x_

    def unsplit(self, x_, 
              modes=None
              ):
        n, b, c, *data_shape = x_.shape
        fft_dims = list(range(-len(data_shape), 0))
        
        Fx_ = torch.fft.fft2(x_, dim=fft_dims, norm='forward') # Fx_ = x_ # 
        
        Fx = torch.zeros((b, c, *data_shape), dtype=Fx_.dtype, device=x_.device)
        k = [torch.fft.fftfreq(n, d=1./n) for n in data_shape[:-1]]
        k += [torch.fft.rfftfreq(data_shape[-1], d=1./data_shape[-1])] if Fx.dtype in [torch.float16, torch.float32, torch.float64] else [torch.fft.fftfreq(data_shape[-1], d=1./data_shape[-1])]
        k = torch.meshgrid(k, indexing='ij')
        k = torch.stack(k)
        k = torch.sqrt(torch.sum(k**2, axis=0)).to(x_.device)
        for i in range(len(modes)):
            k1, k2 = modes[i]
            idx = (k1 <= k) & (k <= k2)
            Fx[..., idx] = Fx_[i, ..., idx]

        x = torch.fft.ifft2(Fx, dim=fft_dims, norm='forward').real
        return x

class EarlyStopping(BaseCallback):
    def __init__(self, 
        criterion=nn.MSELoss(reduction='mean'), # monitor: str,
        min_delta: float = 0.0,
        patience: int = 3,
        verbose: bool = False,
        mode: str = "min",
        strict: bool = True,
        check_finite: bool = True,
        stopping_threshold: Optional[float] = None,
        divergence_threshold: Optional[float] = None,
        check_on_train_epoch_end: Optional[bool] = None,
        log_rank_zero_only: bool = False,
        ):
        super(EarlyStopping, self).__init__()

        self.criterion = criterion
        self.min_delta = min_delta
        self.patience = patience
        self.verbose = verbose
        self.mode = mode
        self.strict = strict
        self.check_finite = check_finite
        self.stopping_threshold = stopping_threshold
        self.divergence_threshold = divergence_threshold
        self.check_on_train_epoch_end = check_on_train_epoch_end
        self.log_rank_zero_only = log_rank_zero_only
        
        self._current_return = []
        self.value_prev = None
        self.patience_count = 0

    def _stopping_criterion(self, value, value_prev):
        flags = []
        if self.mode == 'min':
            flags.append((value_prev - value) < self.min_delta)
        if self.mode == 'max':
            flags.append((value - value_prev) < self.min_delta)

        if self.divergence_threshold is not None:
            flags.append(abs(value) >= self.divergence_threshold)

        if self.stopping_threshold is not None:
            if self.mode == 'min':
                flags.append((value < self.stopping_threshold))
            if self.mode == 'max':
                flags.append((value > self.stopping_threshold))
        return bool(np.sum(flags))
    
    def on_train_batch_end(self, trainer, pl_module, outputs, batch: Any, batch_idx: int) -> None:
        if self.check_on_train_epoch_end:
            return self.on_validation_batch_end(trainer, pl_module, outputs, batch, batch_idx)

    def on_train_epoch_end(self, trainer, pl_module):
        if self.check_on_train_epoch_end:
            return self.on_validation_end(trainer, pl_module)
    
    def on_validation_batch_end(self, trainer, pl_module, outputs, batch: Any, batch_idx: int, dataloader_idx: int = 0) -> None:
        data, target = outputs['batch'] # batch
        value = self.criterion(outputs['output'], target)
        self._current_return.append(value)

    def on_validation_end(self, trainer, pl_module):
        value = np.mean(self._current_return); self._current_return = []
        if self.value_prev is None: 
            self.value_prev = value
            return 

        if self._stopping_criterion(value, self.value_prev):
            if self.patience_count < self.patience:
                self.patience_count += 1
                self.value_prev = value
                return 
            
            trainer.should_stop = True
            self.patience_count = 0
            if self.verbose:
                print(f'EarlyStopping. epoch: {trainer.current_epoch}, value: {value}, delta: {abs(self.value_prev - value)}')
        else:
            self.patience_count = 0
            self.value_prev = value

class CheckPoint(BaseCallback):
    def __init__(self, 
                 ckpt_name:str = None,
                 ckpt_path = './checkpoint/',
                 every_n_epoch: Optional[int] = None,
                 criterion=nn.MSELoss(reduction='mean'), # monitor: str,
                 mode:str = 'min',
                 load_ckpt:bool = True,

                 save_top_k:int=1, 
                 ):
        super(CheckPoint, self).__init__()
        self.ckpt_name = ckpt_name
        self.ckpt_path = ckpt_path 
        os.makedirs(self.ckpt_path, exist_ok=True)

        self.every_n_epoch = every_n_epoch
        self.criterion = criterion
        self.mode = mode 
        self.value_prev = None

        self.load_ckpt = load_ckpt

        self.save_top_k = save_top_k
        self._topk = []

    def _stopping_criterion(self, value, value_prev):
        if self.mode == 'min': return value_prev > value
        if self.mode == 'max': return value_prev < value
    
    def _save(self, trainer, pl_module, suffix):
        name = f'{str(pl_module)}' if self.ckpt_name is None else self.ckpt_name 
        path = os.path.join(self.ckpt_path, name + suffix)
        trainer.save(model=pl_module, path=path)
        return path
    
    def _save_top_k(self, trainer, pl_module, value, epoch):
        def _cmp_key(x):
            return x['value'] if self.mode == 'min' else -x['value']
        
        if not self._topk:
            path = self._save(trainer, pl_module, "_top0")  # must create ckpt at ..._top0.pth
            self._topk = [{"value": value, "epoch": epoch, "path": path}]
            return True
        
        best = self._topk[0]
        if self._stopping_criterion(value, best['value']):
            last_idx = min(len(self._topk), self.save_top_k - 1)
            for i in range(last_idx, 0, -1):
                src_path = self._topk[i - 1]["path"]
                dst_path = os.path.join(self.ckpt_path, f"{self.ckpt_name}_top{i}.pth")
                os.replace(src_path, dst_path)

                if i < len(self._topk):
                    self._topk[i]["value"] = self._topk[i - 1]["value"]
                    self._topk[i]["epoch"] = self._topk[i - 1]["epoch"]
                    self._topk[i]["path"] = dst_path
                else:
                    # When list was shorter than save_top_k, append a new entry
                    self._topk.append({
                        "value": self._topk[i - 1]["value"],
                        "epoch": self._topk[i - 1]["epoch"],
                        "path": dst_path,
                    })
            
            top0_path = os.path.join(self.ckpt_path, f"{self.ckpt_name}_top0.pth") 
            path = self._save(trainer, pl_module, "_top0")
            self._topk[0] = {"value": value, "epoch": epoch, "path": path}
            return True

        return False
    
    def on_fit_start(self, trainer, pl_module):
        if self.load_ckpt:
            ckpt_name = f'{str(pl_module)}' if self.ckpt_name is None else self.ckpt_name
            epoch = trainer.load(model=pl_module, path=os.path.join(self.ckpt_path, ckpt_name))
            if isinstance(epoch, int) and epoch >= 0:
                trainer.current_epoch = epoch

    def on_validation_epoch_start(self, trainer, pl_module):
        # trainer._current_train_return['Checkpoint'] = []
        self._current_return = []
    
    def on_validation_batch_end(self, trainer, pl_module, outputs, batch, batch_idx):
        input, target = outputs['batch']
        output = outputs['output']
        with torch.no_grad():
            v = self.criterion(output, target)
            v = float(v.detach().item()) if torch.is_tensor(v) else float(v)
        self._current_return.append(v)

    def on_validation_epoch_end(self, trainer, pl_module):
        epoch = trainer.current_epoch
        ckpt_name = f'{str(pl_module)}' if self.ckpt_name is None else self.ckpt_name

        ## save every_n_epoch 
        if self.every_n_epoch is not None and epoch % self.every_n_epoch == 0:
            self._save(trainer, pl_module, f'_epoch={epoch}')  # epoch-tagged

        ## save best k case
        value = np.mean(self._current_return) 
        self._save_top_k(trainer, pl_module, value=value, epoch=epoch)



In [12]:
class _CallbackRunner:
    """Pre-binds existing hooks once to minimize per-batch attribute lookups."""
    __slots__ = ("hooks",)
    def __init__(self, callbacks: List[BaseCallback]):
        names = [
            "on_fit_start", "on_fit_end",
            "on_train_start", "on_train_end",
            "on_train_epoch_start", "on_train_epoch_end",
            "on_train_batch_start", "on_train_batch_end",
            "on_validation_start", "on_validation_end",
            "on_validation_batch_start", "on_validation_batch_end",
            "on_validation_epoch_start", "on_validation_epoch_end",
            "on_test_start", "on_test_end",
            "on_test_epoch_start", "on_test_epoch_end",
            "on_test_batch_start", "on_test_batch_end",
        ]
        self.hooks = {n: [] for n in names}
        for cb in callbacks or []:
            for n in names:
                fn = getattr(cb, n, None)
                if fn is not None and callable(fn):
                    self.hooks[n].append(fn)

    def call(self, name: str, *args, **kwargs):
        for fn in self.hooks.get(name, ()):
            fn(*args, **kwargs)

class Trainer:
    def __init__(self, 
                 max_epochs, 
                 check_val_every_n_epochs:int=1,
                 check_val_every_n_iter:int=None,
                 callbacks:List=None, 
                 enable_progress_bar: bool = False, 
                 logger = False,
                 ):
        self.max_epochs = max_epochs
        self._callbacks = _CallbackRunner(callbacks or []) # self.callbacks=callbacks
        self.current_epoch=0
        self.check_val_every_n_epochs = check_val_every_n_epochs
        self.check_val_every_n_iter = check_val_every_n_iter
        self.enable_progress_bar = enable_progress_bar
        self.logger = logger

        self.loss = []
        self.val_loss = []
    
    def progbar_wrapper(self, iterable, total: int, **kwargs: Any):
        if self.enable_progress_bar:
            return tqdm(iterable, total=total, **kwargs)
        return iterable
    
    def fit(self, 
            model, 
            train_dataloaders=None, val_dataloaders=None, 
            datamodule=None, 
            ckpt_path=None
        ):
        self._callbacks.call("on_fit_start", trainer=self, pl_module=model) # for cb in self.callbacks: cb.on_fit_start(trainer=self, pl_module=model)
        if datamodule is not None:
            if not isinstance(datamodule, (list, tuple)): datamodule = [datamodule]
            train_dataloaders, val_dataloaders = [], []
            for dm in datamodule:
                train_dataloaders.append(dm.train_dataloader())
                if dm.val_dataloader(): val_dataloaders.append(dm.val_dataloader())
        if not isinstance(train_dataloaders, (list, tuple)): train_dataloaders = [train_dataloaders]
        if not isinstance(val_dataloaders, (list, tuple)) and val_dataloaders is not None: val_dataloaders = [val_dataloaders]
        
        self.should_stop = False
        while not self.should_stop:
            ## train
            for train_loader in train_dataloaders:
                model.train()
                self.train(model, train_loader)

            ## validation 
            if val_dataloaders and (self.current_epoch % self.check_val_every_n_epochs == 0):
                model.eval()
                for val_loader in val_dataloaders or []:
                    self.validate(model, val_loader)
            
            ## stopping criterion 
            if self.max_epochs is not None and self.current_epoch >= self.max_epochs:
                self.should_stop = True
            self.current_epoch += 1

        # for cb in self.callbacks: cb.on_train_end(trainer=self, pl_module=model)
        self._callbacks.call("on_fit_end", trainer=self, pl_module=model)  
    
    def train(self, model, dataloaders, datamodule=None):
        self._callbacks.call("on_train_start", trainer=self, pl_module=model) 
        if datamodule: dataloaders = datamodule.train_dataloader()

        model.train()
        self._callbacks.call("on_train_epoch_start", trainer=self, pl_module=model) 
        self._current_loss = []
        for batch_idx, batch in self.progbar_wrapper(enumerate(dataloaders), total=len(dataloaders)):
            self._callbacks.call("on_train_batch_start", trainer=self, pl_module=model, batch=batch, batch_idx=batch_idx) 
            outputs = model.training_step(batch, batch_idx)
            if isinstance(outputs, float): self._current_loss.append(outputs)
            self._callbacks.call("on_train_batch_end", trainer=self, pl_module=model, outputs=outputs, batch=batch, batch_idx=batch_idx)
        
        self._callbacks.call("on_train_epoch_end", trainer=self, pl_module=model) 
        self.loss.append(np.mean(self._current_loss))
            
        self._callbacks.call("on_train_end", trainer=self, pl_module=model) 

    def validate(self, model, dataloaders=None, ckpt_path=None, verbose=True, datamodule=None):
        self._callbacks.call("on_validation_start", trainer=self, pl_module=model) 
        if datamodule: dataloaders = datamodule.val_dataloader()

        model.eval()
        with torch.no_grad():
            self._callbacks.call("on_validation_epoch_start", trainer=self, pl_module=model) 
            self._current_val_loss = []
            for batch_idx, batch in self.progbar_wrapper(enumerate(dataloaders), total=len(dataloaders)): # enumerate(dataloaders):
                self._callbacks.call("on_validation_batch_start", trainer=self, pl_module=model, batch=batch, batch_idx=batch_idx) 
                outputs = model.validation_step(batch, batch_idx)

                val_loss = outputs['loss'] if isinstance(outputs, dict) and 'loss' in outputs else outputs
                self._current_val_loss.append(val_loss)
                
                self._callbacks.call("on_validation_batch_end", trainer=self, pl_module=model, outputs=outputs, batch=batch, batch_idx=batch_idx) 
            
            val_loss = np.mean(self._current_val_loss)
            self.val_loss.append(val_loss)
            if self.logger: self.logger.log({'Val/iter/loss':val_loss}, commit=True)
            self._callbacks.call("on_validation_epoch_end", trainer=self, pl_module=model)
        self._callbacks.call("on_validation_end", trainer=self, pl_module=model) 
    
    def test(self, model, dataloaders=None, ckpt_path=None, verbose=True, datamodule=None):
        self._callbacks.call("on_test_start", trainer=self, pl_module=model) 
        if datamodule: dataloaders = datamodule.test_dataloader()

        model.eval()
        with torch.no_grad():
            self._callbacks.call("on_test_epoch_start", trainer=self, pl_module=model) 
            for batch_idx, batch in self.progbar_wrapper(enumerate(dataloaders), total=len(dataloaders)):
                self._callbacks.call("on_test_batch_start", trainer=self, pl_module=model, batch=batch, batch_idx=batch_idx) 
                outputs = model.test_step(batch, batch_idx)
                self._callbacks.call("on_test_batch_end", trainer=self, pl_module=model, outputs=outputs, batch=batch, batch_idx=batch_idx) 
            self._callbacks.call("on_test_epoch_end", trainer=self, pl_module=model) 
        self._callbacks.call("on_test_end", trainer=self, pl_module=model)

    def save(self, model, path):
        checkpoint = {}
        checkpoint['epoch'] = self.current_epoch
        if self.loss: checkpoint['loss'] = self.loss
        if self.val_loss: checkpoint['val_loss'] = self.val_loss
        model.save(path, checkpoint)

    def load(self, model, path):
        model.load(path)
        if os.path.exists(path + '.pth') and os.path.getsize(path + '.pth'):
            checkpoint = torch.load(path + '.pth')
            self.current_epoch = checkpoint['epoch']
            if 'loss' in checkpoint: self.loss = checkpoint['loss']
            if 'val_loss' in checkpoint: self.val_loss = checkpoint['val_loss']
            return self.current_epoch
        return 0

In [13]:
##### set parameters #####
### save parameters ### 
data_dir = config['data'][0]['params']['base_path'] # '../Data/nu=0.001_n=128/'
data_fname = config['data'][0]['params']['dataset_name'] # f'2dHIT_nu=0.001_n=128_T=11.5'

metadata = HIT2dDataset(path=data_dir + 'train/' + data_fname, load_data=False).metadata
print(metadata)

nu = metadata['nu']
dt = metadata['dt']
N = metadata['Nx']

Loaded dataset metadata: dict_keys(['nu', 'Lx', 'Ly', 'dx', 'dy', 'Nx', 'Ny', 'x', 'y', 'dt', 'Nt', 'T', 't0', 't'])
{'nu': 5e-05, 'Lx': 6.234097921967246, 'Ly': 6.234097921967246, 'dx': 0.04908738521234052, 'dy': 0.04908738521234052, 'Nx': 128, 'Ny': 128, 'x': array([0.        , 0.04908739, 0.09817477, 0.14726216, 0.19634954,
       0.24543693, 0.29452431, 0.3436117 , 0.39269908, 0.44178647,
       0.49087385, 0.53996124, 0.58904862, 0.63813601, 0.68722339,
       0.73631078, 0.78539816, 0.83448555, 0.88357293, 0.93266032,
       0.9817477 , 1.03083509, 1.07992247, 1.12900986, 1.17809725,
       1.22718463, 1.27627202, 1.3253594 , 1.37444679, 1.42353417,
       1.47262156, 1.52170894, 1.57079633, 1.61988371, 1.6689711 ,
       1.71805848, 1.76714587, 1.81623325, 1.86532064, 1.91440802,
       1.96349541, 2.01258279, 2.06167018, 2.11075756, 2.15984495,
       2.20893233, 2.25801972, 2.3071071 , 2.35619449, 2.40528188,
       2.45436926, 2.50345665, 2.55254403, 2.60163142, 2.6507188 ,
 

In [14]:
from utils.Losses import (RMSLoss, TKELoss, DissipationLoss, RelambdaLoss, R2, 
                          BSMSE, )
from utils.utilities import HsLoss

##### setup dataset #####
train_dataloaders = [] # datamodules = []
val_dataloaders = []
for i, data_config in enumerate(config['data']):
    transform = CustomTransform()
    normalization = Standardize(normalization_path=data_config['normalization']['normalization_path'])

    idx_leadtime = data_config['idx_leadtime'] if 'idx_leadtime' in data_config else int(data_config['leadtime'] * integral_timescales[nu] / dt) 
    dm = CustomDataModule(**data_config['params'], 
        n_stride=idx_leadtime, 
        # Ndata_train=500, # 500, 
        # Ndata_val=50, 
        # Ndata_test=50,
        # transform=transform,
        normalization=normalization,
        load_data=True,
        )
    stage = data_config['stage']
    dm.setup(stage=stage)
    if stage in ['fit', 'valid']:
        val_dataloaders.append(dm.val_dataloader())
    if stage in ['fit', 'train']:
        train_dataloaders.append(dm.train_dataloader())

test_criterion={
    'l2': nn.MSELoss(), 
    'h1': H1Loss(d=2, reduction='mean'), 
    'h2': HsLoss(k=2, reduction='mean'), # HsLoss(k=2, group=False, size_average=True), 
    
    'fRMS_k<8': BSMSE(kmax=8, dim=(-2, -1), mode='spectral', isRelative=True,),
    'fRMS_8<k<16': BSMSE(kmin=8, kmax=16, dim=(-2, -1), mode='spectral', isRelative=True,),
    'fRMS_k>16': BSMSE(kmin=16, dim=(-2, -1), mode='spectral', isRelative=True,),
    'fRMS_k<kmax': BSMSE(kmax=config['model']['params']['n_modes'][0]//2, dim=(-2, -1), mode='spectral', isRelative=True,),
    'fRMS_k>kmax': BSMSE(kmin=config['model']['params']['n_modes'][0]//2, dim=(-2, -1), mode='spectral', isRelative=True,),
    'fRMS_k>train': BSMSE(kmin=64, dim=(-2, -1), mode='spectral', isRelative=True,),
    'fRMS_k<train': BSMSE(kmax=64, dim=(-2, -1), mode='spectral', isRelative=True,),

    'vor_rms': RMSLoss(dim=2, isRelative=True),
    # 'tke': TKELoss(dim=2, isRelative=True), 
    # 'dissipation': DissipationLoss(nu=nu, dim=2, isRelative=True), 
    # 'R_lambda': RelambdaLoss(nu=nu, dim=2, isRelative=True), 
    'R_squared': R2(), 
    }

Loaded dataset metadata: dict_keys(['nu', 'Lx', 'Ly', 'dx', 'dy', 'Nx', 'Ny', 'x', 'y', 'dt', 'Nt', 'T', 't0', 't'])
Data Loaded from../Data/nu=5e-05_n=512_fDNS=128/train\2dHIT_nu=5e-05_n=512_T=17.0_fDNS=128_0.hdf5. shape: (250, 600, 1, 128, 128), memory: 9830.4 (MB)
Data Loaded from../Data/nu=5e-05_n=512_fDNS=128/train\2dHIT_nu=5e-05_n=512_T=17.0_fDNS=128_1.hdf5. shape: (250, 600, 1, 128, 128), memory: 9830.4 (MB)
Data Loaded. shape: torch.Size([500, 600, 1, 128, 128]), dtype: torch.float32, time: 146.58249187469482 (sec), memory: 19660.8 (MB)
Loaded dataset metadata: dict_keys(['nu', 'Lx', 'Ly', 'dx', 'dy', 'Nx', 'Ny', 'x', 'y', 'dt', 'Nt', 'T', 't0', 't'])
Data Loaded from../Data/nu=5e-05_n=512_fDNS=128/valid\2dHIT_nu=5e-05_n=512_T=17.0_fDNS=128_0.hdf5. shape: (30, 600, 1, 128, 128), memory: 1179.648 (MB)
Data Loaded. shape: torch.Size([30, 600, 1, 128, 128]), dtype: torch.float32, time: 7.764067649841309 (sec), memory: 1179.648 (MB)
Train dataset: 294000
Val dataset: 17640


In [ ]:
from torchinfo import summary

pl_module = CustomModule(
        modelconfig=config['model'], 
        optconfig=config['optimizer'], 
        )
batch_size = config['data'][0]['params']['batch_size']
print(pl_module.device)
summary(
        pl_module.model,
        input_size=(batch_size, 1, N, N),       # batch_size, in_channels, H, W
        col_names=("input_size", "output_size", "num_params", "trainable"),
        depth=5,                           # how many nested modules to show
        verbose=1
    )


cuda
Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Trainable
FNO                                      [64, 1, 128, 128]         [64, 1, 128, 128]         --                        True
├─GridEmbeddingND: 1-1                   [64, 1, 128, 128]         [64, 3, 128, 128]         --                        --
├─ChannelMLP: 1-2                        [64, 3, 128, 128]         [64, 64, 128, 128]        --                        True
│    └─ModuleList: 2-1                   --                        --                        --                        True
│    │    └─Conv1d: 3-1                  [64, 3, 16384]            [64, 256, 16384]          1,024                     True
│    │    └─Conv1d: 3-2                  [64, 256, 16384]          [64, 64, 16384]           16,448                    True
├─FNOBlocks: 1-3                         [64, 64, 128, 128]        [64, 64, 128, 128]        25,977,504                True


Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Trainable
FNO                                      [64, 1, 128, 128]         [64, 1, 128, 128]         --                        True
├─GridEmbeddingND: 1-1                   [64, 1, 128, 128]         [64, 3, 128, 128]         --                        --
├─ChannelMLP: 1-2                        [64, 3, 128, 128]         [64, 64, 128, 128]        --                        True
│    └─ModuleList: 2-1                   --                        --                        --                        True
│    │    └─Conv1d: 3-1                  [64, 3, 16384]            [64, 256, 16384]          1,024                     True
│    │    └─Conv1d: 3-2                  [64, 256, 16384]          [64, 64, 16384]           16,448                    True
├─FNOBlocks: 1-3                         [64, 64, 128, 128]        [64, 64, 128, 128]        25,977,504                True
│    

In [ ]:
from utils.CFDCallback_Spectrum import CFDCallback_Spectrum
from utils.CFDCallback_PhaseError import CFDCallback_PhaseError
from utils.CFDCallback_PDF import CFDCallback_PDF
from utils.CFDCallback_Field import CFDCallback_Field

def main(device='cuda'):
    logger = wandb.init(project=config['name'], 
                        # name=config['name'], 
                        config=config
                        )
    if 'seed' in config: torch.manual_seed(config['seed'])

    data_config = config['data'][0]
    fname=f"2dHIT_{config['model']['name']}_nu={data_config['nu']}"
    fname += f"_T={data_config['idx_leadtime']}" if 'idx_leadtime' in data_config else f"_T={data_config['leadtime']}TL"
    if logger: fname += f'_{logger.name}'

    optconfig = config['optimizer']
    if 'loss_fn' in config: optconfig['loss_fn'] = config['loss_fn']
    if 'lr' in config: optconfig['params']['lr'] = config['lr']
    modelconfig = {**config['model'], }
    modelconfig['params'].update({k: v for k, v in config.items() if k in config['model']['params']})
    

    pl_module = CustomModule(
        modelconfig=config['model'], 
        optconfig=optconfig,  
        )
    # wandb.watch(pl_module, log="all", log_freq=100)
    # pl_module.load('./pretrain/2dHIT_FNOKernel_nu=5e-05_T=0.1TL_scarlet-surf-2_top0', 
    #     load_opt=False, 
    #     freeze=True, 
    #     )
    callbacks = [
            CFDCallback_Spectrum(file_dir='./result/', fname=fname), 
            CFDCallback_PhaseError(file_dir='./result/', fname=fname), 
            CFDCallback_PDF(file_dir='./result/', fname=fname), 
            CFDCallback_Field(file_dir='./result/', fname=fname), 
            CustomCallback(criterions=test_criterion, file_dir='./result/', fname=fname, device=device,
                        ),
            CheckPoint(
                ckpt_name=fname, ckpt_path='./checkpoint/', 
                every_n_epoch = 1,
                criterion=nn.MSELoss(), mode = 'min', 
                load_ckpt=False, 
                ),
            EarlyStopping(criterion=nn.MSELoss(), # neuralop.H1Loss(d=2, reductions='mean'),# 
                            mode='min', min_delta=1e-5, 
                            patience=5, verbose=True, divergence_threshold=1e3, 
                            stopping_threshold=1e-3, 
                            ),
            ]
    trainer = Trainer(
        max_epochs=config['epochs'],
        check_val_every_n_epochs=1,
        # check_val_every_n_iter=30000, 
        enable_progress_bar=True,
        callbacks=callbacks, 
        logger=logger,
        )
    
    trainer.fit(
        model=pl_module,
        train_dataloaders=train_dataloaders, 
        val_dataloaders=val_dataloaders,
        # datamodule = datamodules,
        )
    if logger: wandb.finish()
    return 

In [ ]:
if __name__ == "__main__":
    main()

wandb: Currently logged in as: hjungwon034 (jungwonheo) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


100%|██████████| 275/275 [00:42<00:00,  6.47it/s]
